# Etat de la sécurité alimentaire et de la nutrition dans le monde : une étude des données disponibles sur la FAO pour 2017

Selon le Rapport sur la faim des Nations unies, la faim est le terme utilisé pour définir les périodes où les populations connaissent une insécurité alimentaire grave, c'est-à-dire qu'elles passent des journées entières sans manger par manque d'argent, d'accès à la nourriture ou d'autres ressources.

L'objectif du projet est de dresser un état des lieux de la situation alimentaire dans le monde à partir de données recueillies sur le site de la FAO (*https://www.fao.org/faostat/en/#data*).

Nous utiliserons différentes librairies telles que pandas ou numpy pour manipuler les dataframes et effectuer différents calculs nous permettant d'avoir une vision plus claire de la situation.

## 1. Importation des données et des librairies

In [1]:
# Calculs & dataframes :

import pandas as pd
import numpy as np
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [4]:
# importation des datas

base_dir = r"C:\Users\ynore\Documents\Formation data openclassroom\Projet 4"
sous_nutrition = pd.read_csv(base_dir + "/sous_nutrition.csv")
population = pd.read_csv(base_dir + "/population.csv")
dispo_alimentaire = pd.read_csv(base_dir + "/dispo_alimentaire.csv")
df_aide = pd.read_csv(base_dir + "/aide_alimentaire.csv")

## 2. Calcul des proportions de personne en état de sous-nutrition

### 2.1 Analyse exploratoire des données

La sous-alimentation ou sous-nutrition est un état de manque important de nourriture caractérisé par un apport alimentaire insuffisant pour combler les dépenses énergétiques journalières d'un individu et entraînant des carences nutritionnelles. Chez l'être humain, la sous-nutrition prolongée entraîne des dommages irréversibles aux organes. 

Dans cette partie, nous nous intéressons aux caractéristiques de nos dataframes (df) : types des variables, présence de NaN, nombre de valeurs, etc... Cette analyse préliminaire nous permettra de déterminer en amont si nos df nécessiteront des transformations particulières afin de les analyser plus en profondeur.

In [5]:
print(sous_nutrition.info())
sous_nutrition.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Zone    1218 non-null   object
 1   Année   1218 non-null   object
 2   Valeur  624 non-null    object
dtypes: object(3)
memory usage: 28.7+ KB
None


,Zone,Année,Valeur
count,1218,1218,624
unique,203,6,139
top,Afghanistan,2012-2014,<0.1
freq,6,203,120


In [6]:
print(population.info())
population.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1416 entries, 0 to 1415
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Zone    1416 non-null   object 
 1   Année   1416 non-null   int64  
 2   Valeur  1416 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 33.3+ KB
None


,Année,Valeur
count,1416.000000,1.416000e+03
mean,2015.500000,3.144793e+04
std,1.708428,1.300812e+05
min,2013.000000,7.930000e-01
25%,2014.000000,3.783410e+02
50%,2015.500000,5.126480e+03
75%,2017.000000,1.930666e+04
max,2018.000000,1.427648e+06


In [7]:
population["Zone"].value_counts()

Afghanistan           6
Palestine             6
Nicaragua             6
Niger                 6
Nigéria               6
                     ..
Guinée                6
Guinée équatoriale    6
Guinée-Bissau         6
Guyana                6
Zimbabwe              6
Name: Zone, Length: 236, dtype: int64

In [8]:
# 120 valeurs "<0.1" donc pas de type float/int

print(sous_nutrition.Valeur.value_counts())

<0.1     120
0.3       48
0.4       32
0.2       28
1.5       20
        ... 
3.7        1
24.1       1
189.2      1
190.1      1
8.3        1
Name: Valeur, Length: 139, dtype: int64


In [9]:
# Vérification du nombre de pays dans chaque df

print(len(sous_nutrition.Zone.unique()))
print(len(population.Zone.unique()))

203
236


In [10]:
# Pays pour lesquels les valeurs sont manquantes

absence_data = sous_nutrition.loc[sous_nutrition['Valeur'].isnull()]
absence_data["Zone"].unique()

array(['Allemagne', 'Andorre', 'Antigua-et-Barbuda', 'Argentine',
       'Australie', 'Autriche', 'Azerbaïdjan', 'Bahamas', 'Bahreïn',
       'Bélarus', 'Belgique', 'Bermudes', 'Bhoutan', 'Bosnie-Herzégovine',
       'Brésil', 'Brunéi Darussalam', 'Burundi', 'Canada',
       'Chine - RAS de Hong-Kong', 'Chine, continentale', 'Comores',
       'Croatie', 'Cuba', 'Danemark', 'Djibouti', 'Érythrée', 'Espagne',
       'Estonie', "États-Unis d'Amérique", 'Fédération de Russie',
       'Finlande', 'France', 'Grèce', 'Grenade', 'Groenland', 'Guinée',
       'Guinée équatoriale', 'Guinée-Bissau', 'Hongrie', 'Îles Cook',
       'Îles Marshall', 'Irlande', 'Islande', 'Israël', 'Italie', 'Japon',
       'Kazakhstan', 'Koweït', 'Lettonie', 'Libye', 'Lituanie',
       'Luxembourg', 'Maldives', 'Malte', 'Micronésie (États fédérés de)',
       'Monténégro', 'Nauru', 'Niger', 'Nioué', 'Norvège',
       'Nouvelle-Zélande', 'Ouganda', 'Palaos', 'Palestine',
       'Papouasie-Nouvelle-Guinée', 'Pays-Bas'

### 2.2 Calcul de la proportion de personnes en état de sous-nutrition pour 2017 par pays

Afin de réaliser ce calcul, nous allons devoir sélectionner les données qui nous intéressent pour l'année 2017 dans les 2 df avant d'efefctuer une jointure. Nous utiliseront pour cela les fonction .loc et pd.merge

In [11]:
# On conserve uniquement les valeurs pour 2017 dans les 2 df

sous_nutrition_2017 = sous_nutrition.loc[sous_nutrition['Année'] == '2016-2018']
population_2017 = population.loc[population['Année'] == 2017]

In [12]:
# Renomme une colonne pour ne pas avoir 2 colonnes "Valeur" caractérisant 2 variables différentes

sous_nutrition_2017 = sous_nutrition_2017.rename(columns={"Valeur":"population_sous_nutrition"})

In [13]:
# Transformation de la colonne population_sous_nutrition en float (actuellement "object") afin de pouvoir réaliser les calculs plus tard

sous_nutrition_2017.population_sous_nutrition = pd.to_numeric(sous_nutrition_2017.population_sous_nutrition, errors='coerce')
sous_nutrition_2017.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 203 entries, 4 to 1216
Data columns (total 3 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Zone                       203 non-null    object 
 1   Année                      203 non-null    object 
 2   population_sous_nutrition  84 non-null     float64
dtypes: float64(1), object(2)
memory usage: 6.3+ KB


In [14]:
# Jointure des 2 tables 

comparaison_2017 = pd.merge(population_2017,sous_nutrition_2017, how = 'inner', on = ['Zone']) 
comparaison_2017

,Zone,Année_x,Valeur,Année_y,population_sous_nutrition
0,Afghanistan,2017,36296.113,2016-2018,10.5
1,Afrique du Sud,2017,57009.756,2016-2018,3.1
2,Albanie,2017,2884.169,2016-2018,0.1
3,Algérie,2017,41389.189,2016-2018,1.3
4,Allemagne,2017,82658.409,2016-2018,NaN
...,...,...,...,...,...
198,Venezuela (République bolivarienne du),2017,29402.484,2016-2018,8.0
199,Viet Nam,2017,94600.648,2016-2018,6.5
200,Yémen,2017,27834.819,2016-2018,NaN
201,Zambie,2017,16853.599,2016-2018,NaN


In [15]:
# Clarification de la table obtenue

comparaison_2017 = comparaison_2017.rename(columns={"Année_x":"Annee", "Valeur" : "Population"})
comparaison_2017 = comparaison_2017.drop(columns=["Année_y"])
comparaison_2017

,Zone,Annee,Population,population_sous_nutrition
0,Afghanistan,2017,36296.113,10.5
1,Afrique du Sud,2017,57009.756,3.1
2,Albanie,2017,2884.169,0.1
3,Algérie,2017,41389.189,1.3
4,Allemagne,2017,82658.409,NaN
...,...,...,...,...
198,Venezuela (République bolivarienne du),2017,29402.484,8.0
199,Viet Nam,2017,94600.648,6.5
200,Yémen,2017,27834.819,NaN
201,Zambie,2017,16853.599,NaN


In [16]:
# Mise à l'échelle de la population en millions

comparaison_2017['Population'] = (comparaison_2017['Population'] / 1000).round(decimals = 1)

In [17]:
# Calcul de la proportion par pays

comparaison_2017['proportion'] = round((comparaison_2017['population_sous_nutrition'] / comparaison_2017['Population']) * 100, 1)
comparaison_2017

,Zone,Annee,Population,population_sous_nutrition,proportion
0,Afghanistan,2017,36.3,10.5,28.9
1,Afrique du Sud,2017,57.0,3.1,5.4
2,Albanie,2017,2.9,0.1,3.4
3,Algérie,2017,41.4,1.3,3.1
4,Allemagne,2017,82.7,NaN,NaN
...,...,...,...,...,...
198,Venezuela (République bolivarienne du),2017,29.4,8.0,27.2
199,Viet Nam,2017,94.6,6.5,6.9
200,Yémen,2017,27.8,NaN,NaN
201,Zambie,2017,16.9,NaN,NaN


In [18]:
# Pays pour lesquels les valeurs sont manquantes : on constate qu'on ne peut calculer la proportion de personnes en état de sous-nutrition pour 84 (203-119 = 84) pays soit seulement 41.4% des pays étudiés

comparaison_2017.loc[comparaison_2017['population_sous_nutrition'].isnull()]

,Zone,Annee,Population,population_sous_nutrition,proportion
4,Allemagne,2017,82.7,NaN,NaN
5,Andorre,2017,0.1,NaN,NaN
7,Antigua-et-Barbuda,2017,0.1,NaN,NaN
10,Arménie,2017,2.9,NaN,NaN
11,Australie,2017,24.6,NaN,NaN
...,...,...,...,...,...
196,Uruguay,2017,3.4,NaN,NaN
197,Vanuatu,2017,0.3,NaN,NaN
200,Yémen,2017,27.8,NaN,NaN
201,Zambie,2017,16.9,NaN,NaN


In [19]:
# On conserve le df des 84 pays dont on peut calculer la proportion de personnes en état de sous-nutrition

comparaison = comparaison_2017.dropna(axis=0)
comparaison

,Zone,Annee,Population,population_sous_nutrition,proportion
0,Afghanistan,2017,36.3,10.5,28.9
1,Afrique du Sud,2017,57.0,3.1,5.4
2,Albanie,2017,2.9,0.1,3.4
3,Algérie,2017,41.4,1.3,3.1
6,Angola,2017,29.8,5.8,19.5
...,...,...,...,...,...
187,Togo,2017,7.7,1.6,20.8
192,Turkménistan,2017,5.8,0.2,3.4
195,Ukraine,2017,44.5,1.5,3.4
198,Venezuela (République bolivarienne du),2017,29.4,8.0,27.2


In [20]:
# Somme des populations en état de sous-nutrition

comparaison.population_sous_nutrition.sum()

535.7

In [21]:
# widget pour consulter les lignes du df qui nous intéressent

pays = sorted(list(comparaison['Zone'].unique()))

@interact
def show_per_country(Pays = pays):
    return comparaison.loc[(comparaison['Zone'] == Pays)][['Zone', 'Annee', 'Population', 'population_sous_nutrition',
       'proportion']]

interactive(children=(Dropdown(description='Pays', options=('Afghanistan', 'Afrique du Sud', 'Albanie', 'Algér…

### 2.3 Calcul de la proportion de personnes en état de sous-nutrition au niveau mondial

Ici, il sera plus juste de parler d'échantillon de la population mondiale. En effet, nous ne disposons pour le df sur le nombre de personne en état de sous-nutrition, que des informations concernant 84 pays. Il fait donc sens d'analyser la proportion de personne sous alimentée pour ces 84 pays en rapport avec leur population totale et non avec la population totale des 236 pays dont on a les données.

In [22]:
# calcul de la proportion totale de pers en état de sous-nutrition (sur le df de 84 pays) en 2017

somme_population = comparaison['Population'].sum()
somme_sous_nutrition = round(comparaison['population_sous_nutrition'].sum(),1)
pourcentage_total_sous_nutrition = round((somme_sous_nutrition / somme_population) * 100, 1)

print("La population totale des 84 pays dont les données sont disponibles compte", somme_population, "millions d'individus. Parmi eux,",pourcentage_total_sous_nutrition,"% sont en état de sous-nutrition. Cela représente",somme_sous_nutrition,"millions de personnes")

La population totale des 84 pays dont les données sont disponibles compte 4168.0 millions d'individus. Parmi eux, 12.9 % sont en état de sous-nutrition. Cela représente 535.7 millions de personnes


In [23]:
# Classement des 10 pays comptant la proportion la plus élevée de personnes en état de sous-nutrition en 2017

comparaison.sort_values(by='proportion', ascending=False).head(10)

,Zone,Annee,Population,population_sous_nutrition,proportion
78,Haïti,2017,11.0,5.3,48.2
157,République populaire démocratique de Corée,2017,25.4,12.0,47.2
108,Madagascar,2017,25.6,10.5,41.0
103,Libéria,2017,4.7,1.8,38.3
100,Lesotho,2017,2.1,0.8,38.1
183,Tchad,2017,15.0,5.7,38.0
161,Rwanda,2017,12.0,4.2,35.0
186,Timor-Leste,2017,1.2,0.4,33.3
121,Mozambique,2017,28.6,9.4,32.9
0,Afghanistan,2017,36.3,10.5,28.9


In [24]:
# Classement des pays comptant le + de personnes en état de sous-nutrition en 2017

comparaison.sort_values(by='population_sous_nutrition', ascending=False)

,Zone,Annee,Population,population_sous_nutrition,proportion
84,Inde,2017,1338.7,190.1,14.2
136,Pakistan,2017,207.9,24.8,11.9
85,Indonésie,2017,264.7,23.6,8.9
128,Nigéria,2017,190.9,22.8,11.9
16,Bangladesh,2017,159.7,21.5,13.5
...,...,...,...,...,...
59,Eswatini,2017,1.1,0.2,18.2
45,Costa Rica,2017,4.9,0.2,4.1
192,Turkménistan,2017,5.8,0.2,3.4
29,Bulgarie,2017,7.1,0.2,2.8


## 3. La disponibilité alimentaire

La disponibilité des produits alimentaires est une condition de la sécurité alimentaire. Cela veut dire que dans un pays, une région, une localité, les produits alimentaires sont physiquement disponibles en quantité suffisante pour assurer la sécurité alimentaire de ses habitants. La provenance de ces aliments peut être soit locale, soit nationale ou d’importation étrangère.

### 3.1 Analyse exploratoire des données

In [25]:
print(dispo_alimentaire.info())
dispo_alimentaire.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15605 entries, 0 to 15604
Data columns (total 18 columns):
 #   Column                                                         Non-Null Count  Dtype  
---  ------                                                         --------------  -----  
 0   Zone                                                           15605 non-null  object 
 1   Produit                                                        15605 non-null  object 
 2   Origine                                                        15605 non-null  object 
 3   Aliments pour animaux                                          2720 non-null   float64
 4   Autres Utilisations                                            5496 non-null   float64
 5   Disponibilité alimentaire (Kcal/personne/jour)                 14241 non-null  float64
 6   Disponibilité alimentaire en quantité (kg/personne/an)         14015 non-null  float64
 7   Disponibilité de matière grasse en quantité (g/personne/jo

,Aliments pour animaux,Autres Utilisations,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Semences,Traitement,Variation de stock
count,2720.000000,5496.000000,14241.000000,14015.000000,11794.000000,11561.000000,15382.000000,12226.000000,14852.000000,14015.000000,4278.000000,9180.000000,2091.000000,2292.000000,6776.000000
mean,479.501838,157.391376,34.789832,8.719368,1.283111,1.223608,640.293460,110.596925,87.264543,347.931359,106.053763,1090.379085,73.974653,961.905323,-15.407615
std,4240.119637,5076.785816,107.287655,24.618223,3.680399,3.598686,9067.267153,1053.318990,717.372714,4475.704458,1113.100416,12067.344094,528.069224,10381.795904,549.834540
min,0.000000,0.000000,-21.000000,-1.930000,-0.030000,-0.370000,-3430.000000,-41.000000,-201.000000,-246.000000,0.000000,0.000000,0.000000,-19.000000,-39863.000000
25%,0.000000,0.000000,0.000000,0.060000,0.010000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000
50%,4.000000,0.000000,4.000000,0.830000,0.080000,0.100000,7.000000,0.000000,2.000000,5.000000,4.000000,22.000000,2.000000,6.000000,0.000000
75%,74.000000,4.000000,21.000000,5.190000,0.630000,0.660000,76.750000,9.000000,18.000000,52.000000,26.000000,191.250000,17.000000,69.000000,0.000000
max,150000.000000,347309.000000,1711.000000,430.760000,60.760000,54.970000,739267.000000,42797.000000,63381.000000,426850.000000,55047.000000,739267.000000,17060.000000,326711.000000,5284.000000


In [26]:
dispo_alimentaire.sample(10)

,Zone,Produit,Origine,Aliments pour animaux,Autres Utilisations,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Semences,Traitement,Variation de stock
14528,Vanuatu,Arachides Decortiquees,vegetale,NaN,0.0,112.0,7.98,9.31,4.84,2.0,0.0,0.0,2.0,0.0,2.0,NaN,NaN,NaN
5827,Géorgie,Animaux Aquatiques Autre,animale,NaN,0.0,0.0,0.00,0.00,0.00,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN
1329,Bangladesh,Racines nda,vegetale,NaN,NaN,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
7565,Koweït,Animaux Aquatiques Autre,animale,NaN,0.0,0.0,0.03,0.00,0.01,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN
3749,Croatie,Animaux Aquatiques Autre,animale,NaN,0.0,0.0,0.08,0.00,0.01,0.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN,0.0
2871,Canada,"Plantes Oleiferes, Autre",vegetale,NaN,41.0,1.0,0.05,0.06,0.03,21.0,707.0,74.0,2.0,22.0,754.0,15.0,30.0,-100.0
13202,Suisse,Huile de Tournesol,vegetale,NaN,3.0,124.0,5.13,14.06,0.00,45.0,1.0,41.0,41.0,NaN,5.0,NaN,NaN,0.0
9162,Mongolie,"Beurre, Ghee",animale,NaN,NaN,8.0,0.42,0.93,0.01,1.0,0.0,0.0,1.0,NaN,1.0,NaN,NaN,NaN
4822,Finlande,Raisin,vegetale,NaN,NaN,9.0,4.89,0.03,0.09,27.0,0.0,27.0,27.0,0.0,NaN,NaN,NaN,0.0
2510,Bénin,Plantes Aquatiques,animale,NaN,0.0,0.0,0.00,0.00,0.00,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN


### 3.2 Calcul de la disponibilité alimentaire totale (d'origine végétale & animale)

Notre analyse se concentra essentiellement sur la disponibilité alimentaire en Kcal afin de rester succint dans nos résultats. Néanmoins, et si l'on souhaitait être exhaustif, une alimentation équilibrée suppose également des apports nutritionnels en protéines et en matière grasse qui ne peuvent être pris indépendemment les uns des autres. On admettra également que les besoins caloriques quotidien d'un individu se situent à 2400 Kcal (moyenne homme/femme permettant une activité physique modérée au cours de la journée).

In [27]:
# Suppression des colonnes inutiles

dispo_alimentaire_clear = dispo_alimentaire.drop(columns=[
    "Aliments pour animaux", 
    "Origine",
    "Autres Utilisations", 
    "Disponibilité alimentaire en quantité (kg/personne/an)",
    "Disponibilité de matière grasse en quantité (g/personne/jour)",
    "Disponibilité de protéines en quantité (g/personne/jour)",
    "Disponibilité intérieure", "Exportations - Quantité", 
    "Importations - Quantité", 
    "Nourriture", 
    "Pertes",
    "Production",
    "Semences",
    "Traitement",
    "Variation de stock"])
dispo_alimentaire_clear

,Zone,Produit,Disponibilité alimentaire (Kcal/personne/jour)
0,Afghanistan,Abats Comestible,5.0
1,Afghanistan,"Agrumes, Autres",1.0
2,Afghanistan,Aliments pour enfants,1.0
3,Afghanistan,Ananas,0.0
4,Afghanistan,Bananes,4.0
...,...,...,...
15600,Îles Salomon,Viande de Suides,45.0
15601,Îles Salomon,Viande de Volailles,11.0
15602,Îles Salomon,"Viande, Autre",0.0
15603,Îles Salomon,Vin,0.0


In [28]:
# Uniformisation des noms des pays des 2 tables (problème identifié sous excel)

population_2017['Zone'] = population_2017['Zone'].replace([
    "Royaume-Uni de Grande-Bretagne et d'Irlande du Nord",
    "Tchéquie"],
    ["Royaume-Uni","Tchéquie (la)"])

C:\Users\ynore\AppData\Local\Temp/ipykernel_15640/3692373066.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  population_2017['Zone'] = population_2017['Zone'].replace([


In [29]:
# Aggrégation des disponibilités alimentaires par pays

table_dispo = pd.pivot_table(data=dispo_alimentaire_clear,index=['Zone'],aggfunc="sum")
table_dispo

,Disponibilité alimentaire (Kcal/personne/jour)
Zone,
Afghanistan,2087.0
Afrique du Sud,3020.0
Albanie,3188.0
Algérie,3293.0
Allemagne,3503.0
...,...
Émirats arabes unis,3275.0
Équateur,2346.0
États-Unis d'Amérique,3682.0


L'aggrégation par pays nous permet d'aggréger ( = additionner) les disponibilités de chaque aliment dans un même pays. La somme cumulée de ces disponibilités nous permettent ainsi de déterminer la disponibilité alimentaire totale par habitant/jour/Kcal.

In [30]:
# Jointure des 2 tables

dispo_alimentaire_2017 = pd.merge(population_2017,table_dispo, how='inner', on=['Zone'])
dispo_alimentaire_2017 = dispo_alimentaire_2017.rename(columns={"Valeur" : "Population"})
dispo_alimentaire_2017

,Zone,Année,Population,Disponibilité alimentaire (Kcal/personne/jour)
0,Afghanistan,2017,36296.113,2087.0
1,Afrique du Sud,2017,57009.756,3020.0
2,Albanie,2017,2884.169,3188.0
3,Algérie,2017,41389.189,3293.0
4,Allemagne,2017,82658.409,3503.0
...,...,...,...,...
169,Venezuela (République bolivarienne du),2017,29402.484,2633.0
170,Viet Nam,2017,94600.648,2744.0
171,Yémen,2017,27834.819,2217.0
172,Zambie,2017,16853.599,1924.0


In [31]:
# Remise à l'échelle de la population

dispo_alimentaire_2017['Population'] = (dispo_alimentaire_2017['Population'] * 1000).round(decimals = 1)
dispo_alimentaire_2017

,Zone,Année,Population,Disponibilité alimentaire (Kcal/personne/jour)
0,Afghanistan,2017,36296113.0,2087.0
1,Afrique du Sud,2017,57009756.0,3020.0
2,Albanie,2017,2884169.0,3188.0
3,Algérie,2017,41389189.0,3293.0
4,Allemagne,2017,82658409.0,3503.0
...,...,...,...,...
169,Venezuela (République bolivarienne du),2017,29402484.0,2633.0
170,Viet Nam,2017,94600648.0,2744.0
171,Yémen,2017,27834819.0,2217.0
172,Zambie,2017,16853599.0,1924.0


Ici, la jointure des 2 tables nous permet de calculer la disponibilité alimentaire totale théorique en Kcal dans chaque pays. On constate en observant la table obtenue que la la disponibilité alimentaire par personne/jour varie grandement d'un pays à l'autre et que certain pays (notamment l'Allemgane par exemple) possède une disopibilité alimentaire largement supérieure aux besoins quotidien d'une personne (~2400 Kcal/j). 

In [32]:
# Calcul des dispo alimentaire totales par pays par jour

dispo_alimentaire_2017['dispo_kcal_pays'] = round((dispo_alimentaire_2017['Population'] * dispo_alimentaire_2017['Disponibilité alimentaire (Kcal/personne/jour)']), 1)
dispo_alimentaire_2017

,Zone,Année,Population,Disponibilité alimentaire (Kcal/personne/jour),dispo_kcal_pays
0,Afghanistan,2017,36296113.0,2087.0,7.574999e+10
1,Afrique du Sud,2017,57009756.0,3020.0,1.721695e+11
2,Albanie,2017,2884169.0,3188.0,9.194731e+09
3,Algérie,2017,41389189.0,3293.0,1.362946e+11
4,Allemagne,2017,82658409.0,3503.0,2.895524e+11
...,...,...,...,...,...
169,Venezuela (République bolivarienne du),2017,29402484.0,2633.0,7.741674e+10
170,Viet Nam,2017,94600648.0,2744.0,2.595842e+11
171,Yémen,2017,27834819.0,2217.0,6.170979e+10
172,Zambie,2017,16853599.0,1924.0,3.242632e+10


In [33]:
# Calcul du nombre de personnes qui pourraient être nourries sur une base 2400 kcal/j

pop_totale = dispo_alimentaire_2017["Population"].sum()
kcal_total = dispo_alimentaire_2017["dispo_kcal_pays"].sum()
kcal_mini = 2400
pers_nourrie = round((kcal_total/kcal_mini))

print(pers_nourrie,"individus pourraient être nourries sur une base de 2400 Kcal par jour si l'alimentation disponible était équitablement répartie dans le monde")

# calcul du % de la population totale qui pourrait être nourrie par la disponibilité totale
proportion_personne_nourrie = round((pers_nourrie / pop_totale) * 100,1)

print("Cela représente", proportion_personne_nourrie,"% de la population actuelle")

8825901145 individus pourraient être nourries sur une base de 2400 Kcal par jour si l'alimentation disponible était équitablement répartie dans le monde
Cela représente 119.8 % de la population actuelle


Compte tenu du nombre d'habitants de la planète et des besoins énergétiques de chacun, nous pourrions nourrir la popualtion mondiale avec les seules infrastructures agricoles déjà existantes. Alors si les ressources existent, d'où vient le problème ? Mauvaise répartition des disponibilités ? Pertes et gâchis de nourriture ? Modes de consommation inadaptés ? 

### 3.3 Calcul de la disponibilité alimentaire végétale

In [34]:
# Filtre sur les disponibilités végétales

dispo_alimentaire_vegetale = dispo_alimentaire_clear.loc[dispo_alimentaire['Origine'] == 'vegetale']

In [35]:
# Agrégation par pays

table2 = pd.pivot_table(data=dispo_alimentaire_vegetale,index=['Zone'],aggfunc="sum")
table2

,Disponibilité alimentaire (Kcal/personne/jour)
Zone,
Afghanistan,1871.0
Afrique du Sud,2533.0
Albanie,2203.0
Algérie,2915.0
Allemagne,2461.0
...,...
Émirats arabes unis,2718.0
Équateur,1732.0
États-Unis d'Amérique,2698.0


In [36]:
# Jointure des 2 tables

dispo_vegetale = pd.merge(population_2017,table2, how='inner', on=['Zone'])
dispo_vegetale = dispo_vegetale.rename(columns={"Valeur" : "Population"})
dispo_vegetale

,Zone,Année,Population,Disponibilité alimentaire (Kcal/personne/jour)
0,Afghanistan,2017,36296.113,1871.0
1,Afrique du Sud,2017,57009.756,2533.0
2,Albanie,2017,2884.169,2203.0
3,Algérie,2017,41389.189,2915.0
4,Allemagne,2017,82658.409,2461.0
...,...,...,...,...
169,Venezuela (République bolivarienne du),2017,29402.484,2157.0
170,Viet Nam,2017,94600.648,2169.0
171,Yémen,2017,27834.819,2028.0
172,Zambie,2017,16853.599,1818.0


In [37]:
# Remise à l'échelle de la population

dispo_vegetale['Population'] = (dispo_vegetale['Population'] * 1000).round(decimals = 1)

In [38]:
# Calcul des dispo alimentaire vegetales totales par pays par jour

dispo_vegetale['dispo_kcal_vege'] = round((dispo_vegetale['Population'] * dispo_vegetale['Disponibilité alimentaire (Kcal/personne/jour)']), 1)
dispo_vegetale

,Zone,Année,Population,Disponibilité alimentaire (Kcal/personne/jour),dispo_kcal_vege
0,Afghanistan,2017,36296113.0,1871.0,6.791003e+10
1,Afrique du Sud,2017,57009756.0,2533.0,1.444057e+11
2,Albanie,2017,2884169.0,2203.0,6.353824e+09
3,Algérie,2017,41389189.0,2915.0,1.206495e+11
4,Allemagne,2017,82658409.0,2461.0,2.034223e+11
...,...,...,...,...,...
169,Venezuela (République bolivarienne du),2017,29402484.0,2157.0,6.342116e+10
170,Viet Nam,2017,94600648.0,2169.0,2.051888e+11
171,Yémen,2017,27834819.0,2028.0,5.644901e+10
172,Zambie,2017,16853599.0,1818.0,3.063984e+10


In [39]:
# Calcul du nombre de personnes qui pourraient nourries sur une base 2400 kcal/j

pop_totale_vege = dispo_vegetale["Population"].sum()
kcal_total_vege = dispo_vegetale["dispo_kcal_vege"].sum()
pers_nourrie_vege = round((kcal_total_vege/kcal_mini))
print(pers_nourrie_vege,"individus pourraient être nourries sur une base de 2400 Kcal par jour si l'alimentation végétale était équitablement répartie dans le monde")

# Calcul du % de la population totale qui pourrait être nourrie par la disponibilité végétale

proportion_personne_nourrie_vege = round((pers_nourrie_vege / pop_totale_vege) * 100,1)
print("Cela représente", proportion_personne_nourrie_vege,"% de la population actuelle")

7270628925 individus pourraient être nourries sur une base de 2400 Kcal par jour si l'alimentation végétale était équitablement répartie dans le monde
Cela représente 98.7 % de la population actuelle


### 3.4 Calcul de la disponiblité alimentaire par habitant pour 2017

In [40]:
# Clarification du dataframe et création d'un tableau croisé dynamique

df_dispo = dispo_alimentaire.drop(columns=[
    "Aliments pour animaux", 
    "Produit", 
    "Autres Utilisations",
    "Disponibilité intérieure", 
    "Exportations - Quantité", 
    "Importations - Quantité", 
    "Nourriture", 
    "Pertes",
    "Production",
    "Semences",
    "Traitement",
    "Variation de stock"])

table_dispo_aliment = pd.pivot_table(data=df_dispo,index=['Zone'],aggfunc="sum")
table_dispo_aliment

,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour)
Zone,,,,
Afghanistan,2087.0,351.41,33.50,58.26
Afrique du Sud,3020.0,564.80,83.58,85.33
Albanie,3188.0,1095.54,106.48,111.37
Algérie,3293.0,809.13,76.48,91.92
Allemagne,3503.0,957.52,141.62,101.39
...,...,...,...,...
Émirats arabes unis,3275.0,700.24,87.16,104.51
Équateur,2346.0,544.91,94.64,61.30
États-Unis d'Amérique,3682.0,1002.39,161.37,109.42


In [41]:
# jointure des 2 tables et clarification de la table obtenue

comparaison_dispo = pd.merge(population_2017,table_dispo_aliment, left_on= "Zone", right_on= "Zone")
comparaison_dispo = comparaison_dispo.rename(columns={"Valeur" : "Population"})
comparaison_dispo['Population'] = (comparaison_dispo['Population'] * 1000).round(decimals = 1)
comparaison_dispo

,Zone,Année,Population,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour)
0,Afghanistan,2017,36296113.0,2087.0,351.41,33.50,58.26
1,Afrique du Sud,2017,57009756.0,3020.0,564.80,83.58,85.33
2,Albanie,2017,2884169.0,3188.0,1095.54,106.48,111.37
3,Algérie,2017,41389189.0,3293.0,809.13,76.48,91.92
4,Allemagne,2017,82658409.0,3503.0,957.52,141.62,101.39
...,...,...,...,...,...,...,...
169,Venezuela (République bolivarienne du),2017,29402484.0,2633.0,652.86,82.44,72.96
170,Viet Nam,2017,94600648.0,2744.0,574.84,72.11,81.61
171,Yémen,2017,27834819.0,2217.0,375.41,47.81,58.90
172,Zambie,2017,16853599.0,1924.0,350.12,42.05,55.22


In [42]:
# Classement des pays ayant le plus faible taux de disponibilité alimentaire Kcal par personne et par jour

comparaison_dispo.drop(columns= [
    'Disponibilité alimentaire en quantité (kg/personne/an)',
    'Disponibilité de matière grasse en quantité (g/personne/jour)',
    'Disponibilité de protéines en quantité (g/personne/jour)']).sort_values(by='Disponibilité alimentaire (Kcal/personne/jour)', ascending=True).head(10)

,Zone,Année,Population,Disponibilité alimentaire (Kcal/personne/jour)
131,République centrafricaine,2017,4596023.0,1879.0
172,Zambie,2017,16853599.0,1924.0
97,Madagascar,2017,25570512.0,2056.0
0,Afghanistan,2017,36296113.0,2087.0
70,Haïti,2017,10982366.0,2089.0
136,République populaire démocratique de Corée,2017,25429825.0,2093.0
157,Tchad,2017,15016753.0,2109.0
173,Zimbabwe,2017,14236595.0,2113.0
120,Ouganda,2017,41166588.0,2126.0
55,Éthiopie,2017,106399924.0,2129.0


In [43]:
# Classement des pays ayant le plus haut taux de disponibilité alimentaire Kcal par personne et par jour

comparaison_dispo.drop(columns= [
    'Disponibilité alimentaire en quantité (kg/personne/an)',
    'Disponibilité de matière grasse en quantité (g/personne/jour)',
    'Disponibilité de protéines en quantité (g/personne/jour)']).sort_values(by='Disponibilité alimentaire (Kcal/personne/jour)', ascending=False).head(10)

,Zone,Année,Population,Disponibilité alimentaire (Kcal/personne/jour)
11,Autriche,2017,8819901.0,3770.0
17,Belgique,2017,11419748.0,3737.0
165,Turquie,2017,81116450.0,3708.0
54,États-Unis d'Amérique,2017,325084756.0,3682.0
80,Israël,2017,8243848.0,3610.0
78,Irlande,2017,4753279.0,3602.0
81,Italie,2017,60673701.0,3578.0
95,Luxembourg,2017,591910.0,3540.0
47,Égypte,2017,96442591.0,3518.0
4,Allemagne,2017,82658409.0,3503.0


In [44]:
# Classement des pays ayant le plus faible taux de disponibilité alimentaire en quantité par personne et par an

comparaison_dispo.drop(columns = [
    'Disponibilité alimentaire (Kcal/personne/jour)',
    'Disponibilité de matière grasse en quantité (g/personne/jour)',
    'Disponibilité de protéines en quantité (g/personne/jour)']).sort_values(by='Disponibilité alimentaire en quantité (kg/personne/an)', ascending=True).head(10)

,Zone,Année,Population,Disponibilité alimentaire en quantité (kg/personne/an)
157,Tchad,2017,15016753.0,319.45
173,Zimbabwe,2017,14236595.0,336.59
172,Zambie,2017,16853599.0,350.12
0,Afghanistan,2017,36296113.0,351.41
55,Éthiopie,2017,106399924.0,354.55
68,Guinée-Bissau,2017,1828145.0,358.48
14,Bangladesh,2017,159685424.0,364.54
160,Timor-Leste,2017,1243258.0,369.10
171,Yémen,2017,27834819.0,375.41
146,Sénégal,2017,15419355.0,376.72


In [45]:
# Classement des pays ayant le plus faible taux de disponibilité en matière grasse en g/personne/jour (sachant que les besoins moyens se situent à 100g/j)

comparaison_dispo.drop(columns = [
    'Disponibilité alimentaire (Kcal/personne/jour)',
    'Disponibilité alimentaire en quantité (kg/personne/an)',
    'Disponibilité de protéines en quantité (g/personne/jour)']).sort_values(by='Disponibilité de matière grasse en quantité (g/personne/jour)', ascending=True).head(10)

,Zone,Année,Population,Disponibilité de matière grasse en quantité (g/personne/jour)
140,Rwanda,2017,11980961.0,23.77
97,Madagascar,2017,25570512.0,24.49
55,Éthiopie,2017,106399924.0,25.89
14,Bangladesh,2017,159685424.0,29.92
90,Lesotho,2017,2091534.0,32.26
0,Afghanistan,2017,36296113.0,33.50
29,Cambodge,2017,16009409.0,35.93
136,République populaire démocratique de Corée,2017,25429825.0,36.41
134,République démocratique populaire lao,2017,6953035.0,38.86
99,Malawi,2017,17670196.0,41.17


In [46]:
# Classement des pays ayant le plus faible taux de disponibilité en protéines grasse en g/personne/jour (sachant que les besoins moyens se situent à 50g/j)

comparaison_dispo.drop(columns = [
    'Disponibilité alimentaire (Kcal/personne/jour)',
    'Disponibilité alimentaire en quantité (kg/personne/an)',
    'Disponibilité de matière grasse en quantité (g/personne/jour)']).sort_values(by='Disponibilité de protéines en quantité (g/personne/jour)', ascending=True).head(10)

,Zone,Année,Population,Disponibilité de protéines en quantité (g/personne/jour)
93,Libéria,2017,4702226.0,37.66
68,Guinée-Bissau,2017,1828145.0,44.05
109,Mozambique,2017,28649018.0,45.68
131,République centrafricaine,2017,4596023.0,46.04
97,Madagascar,2017,25570512.0,46.69
70,Haïti,2017,10982366.0,47.70
173,Zimbabwe,2017,14236595.0,48.32
39,Congo,2017,5110695.0,51.41
120,Ouganda,2017,41166588.0,52.64
145,Sao Tomé-et-Principe,2017,207089.0,53.10


## 4. Valeur énergétique des aliments

La valeur énergétique correspond à l’apport calorique de l’aliment, exprimé en kilocalories (kcal) ou  en kilojoules (kJ).Les besoins quotidiens en calories varient selon le sexe, l'âge, la corpulence, la dépense physique et l'état de santé. Un aliment est considéré comme riche en calories quand il dépasse 300 calories pour 100 g. Comme nous l'avons vu plus haut, il ne faut cependant pas se contenter de la seule appréciation des calories, car la signification énergétique d’un produit alimentaire varie selon sa composition en nutriments (glucides, lipides, protéines, vitamines…).

### 4.1 Calcul de la valeur énergétique (ratio énergie / poids) des produits

In [47]:
# Clarification de la table

dispo_alimentaire_clear = dispo_alimentaire.drop(columns=[
    "Aliments pour animaux", 
    "Autres Utilisations", 
    "Disponibilité de matière grasse en quantité (g/personne/jour)",
    "Disponibilité intérieure", "Exportations - Quantité", 
    "Importations - Quantité", 
    "Nourriture", 
    "Pertes",
    "Production",
    "Semences",
    "Traitement",
    "Variation de stock"])
dispo_alimentaire_clear

,Zone,Produit,Origine,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de protéines en quantité (g/personne/jour)
0,Afghanistan,Abats Comestible,animale,5.0,1.72,0.77
1,Afghanistan,"Agrumes, Autres",vegetale,1.0,1.29,0.02
2,Afghanistan,Aliments pour enfants,vegetale,1.0,0.06,0.03
3,Afghanistan,Ananas,vegetale,0.0,0.00,NaN
4,Afghanistan,Bananes,vegetale,4.0,2.70,0.05
...,...,...,...,...,...,...
15600,Îles Salomon,Viande de Suides,animale,45.0,4.70,1.41
15601,Îles Salomon,Viande de Volailles,animale,11.0,3.34,1.14
15602,Îles Salomon,"Viande, Autre",animale,0.0,0.06,0.04
15603,Îles Salomon,Vin,vegetale,0.0,0.07,NaN


In [48]:
# Jointure des tables sur Zone + remise à l'échelle de la population

ratio_population = pd.merge(dispo_alimentaire_clear, population_2017, how='left', on=['Zone'])
ratio_population['Valeur'] = (ratio_population['Valeur'] * 1000).round(decimals = 1)
ratio_population = ratio_population.rename(columns={"Valeur" : "Population"})
ratio_population

,Zone,Produit,Origine,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de protéines en quantité (g/personne/jour),Année,Population
0,Afghanistan,Abats Comestible,animale,5.0,1.72,0.77,2017,36296113.0
1,Afghanistan,"Agrumes, Autres",vegetale,1.0,1.29,0.02,2017,36296113.0
2,Afghanistan,Aliments pour enfants,vegetale,1.0,0.06,0.03,2017,36296113.0
3,Afghanistan,Ananas,vegetale,0.0,0.00,NaN,2017,36296113.0
4,Afghanistan,Bananes,vegetale,4.0,2.70,0.05,2017,36296113.0
...,...,...,...,...,...,...,...,...
15600,Îles Salomon,Viande de Suides,animale,45.0,4.70,1.41,2017,636039.0
15601,Îles Salomon,Viande de Volailles,animale,11.0,3.34,1.14,2017,636039.0
15602,Îles Salomon,"Viande, Autre",animale,0.0,0.06,0.04,2017,636039.0
15603,Îles Salomon,Vin,vegetale,0.0,0.07,NaN,2017,636039.0


In [49]:
# Calcul de la disponibilité calorique totale par pays et par personne par pays

kcal_total_an = round((ratio_population['Population'] * ratio_population['Disponibilité alimentaire (Kcal/personne/jour)']) * 365, 1)
kcal_pers_an = round((ratio_population['Population'] * ratio_population['Disponibilité alimentaire (Kcal/personne/jour)']), 1)

ratio_population['dispo Kcal/pers/an'], ratio_population['dispo Kcal/total/an']  = [kcal_pers_an, kcal_total_an]
ratio_population

,Zone,Produit,Origine,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de protéines en quantité (g/personne/jour),Année,Population,dispo Kcal/pers/an,dispo Kcal/total/an
0,Afghanistan,Abats Comestible,animale,5.0,1.72,0.77,2017,36296113.0,181480565.0,6.624041e+10
1,Afghanistan,"Agrumes, Autres",vegetale,1.0,1.29,0.02,2017,36296113.0,36296113.0,1.324808e+10
2,Afghanistan,Aliments pour enfants,vegetale,1.0,0.06,0.03,2017,36296113.0,36296113.0,1.324808e+10
3,Afghanistan,Ananas,vegetale,0.0,0.00,NaN,2017,36296113.0,0.0,0.000000e+00
4,Afghanistan,Bananes,vegetale,4.0,2.70,0.05,2017,36296113.0,145184452.0,5.299232e+10
...,...,...,...,...,...,...,...,...,...,...
15600,Îles Salomon,Viande de Suides,animale,45.0,4.70,1.41,2017,636039.0,28621755.0,1.044694e+10
15601,Îles Salomon,Viande de Volailles,animale,11.0,3.34,1.14,2017,636039.0,6996429.0,2.553697e+09
15602,Îles Salomon,"Viande, Autre",animale,0.0,0.06,0.04,2017,636039.0,0.0,0.000000e+00
15603,Îles Salomon,Vin,vegetale,0.0,0.07,NaN,2017,636039.0,0.0,0.000000e+00


In [50]:
# Calcul des quantités totale disponibles pour chaque produit (en KG) par an dans le pays

ratio_population['dispo kg/total/an']= round((ratio_population['Population'] * ratio_population["Disponibilité alimentaire en quantité (kg/personne/an)"]), 1)
ratio_population

,Zone,Produit,Origine,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de protéines en quantité (g/personne/jour),Année,Population,dispo Kcal/pers/an,dispo Kcal/total/an,dispo kg/total/an
0,Afghanistan,Abats Comestible,animale,5.0,1.72,0.77,2017,36296113.0,181480565.0,6.624041e+10,62429314.4
1,Afghanistan,"Agrumes, Autres",vegetale,1.0,1.29,0.02,2017,36296113.0,36296113.0,1.324808e+10,46821985.8
2,Afghanistan,Aliments pour enfants,vegetale,1.0,0.06,0.03,2017,36296113.0,36296113.0,1.324808e+10,2177766.8
3,Afghanistan,Ananas,vegetale,0.0,0.00,NaN,2017,36296113.0,0.0,0.000000e+00,0.0
4,Afghanistan,Bananes,vegetale,4.0,2.70,0.05,2017,36296113.0,145184452.0,5.299232e+10,97999505.1
...,...,...,...,...,...,...,...,...,...,...,...
15600,Îles Salomon,Viande de Suides,animale,45.0,4.70,1.41,2017,636039.0,28621755.0,1.044694e+10,2989383.3
15601,Îles Salomon,Viande de Volailles,animale,11.0,3.34,1.14,2017,636039.0,6996429.0,2.553697e+09,2124370.3
15602,Îles Salomon,"Viande, Autre",animale,0.0,0.06,0.04,2017,636039.0,0.0,0.000000e+00,38162.3
15603,Îles Salomon,Vin,vegetale,0.0,0.07,NaN,2017,636039.0,0.0,0.000000e+00,44522.7


In [51]:
# Calcul de la disponibilité totale de protéines en KG  par personne pour chaque produit par pays et par an

kg_prot_pers = round(ratio_population["Disponibilité de protéines en quantité (g/personne/jour)"] / 1000,6)
ratio_population["Disponibilité de protéines en quantité (KG/personne/jour)"] = kg_prot_pers

prot_total_an = round((ratio_population['Population'] * ratio_population["Disponibilité de protéines en quantité (KG/personne/jour)"]) * 365, 1)

ratio_population['dispo prot_total/kg/an']  = prot_total_an
ratio_population

,Zone,Produit,Origine,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de protéines en quantité (g/personne/jour),Année,Population,dispo Kcal/pers/an,dispo Kcal/total/an,dispo kg/total/an,Disponibilité de protéines en quantité (KG/personne/jour),dispo prot_total/kg/an
0,Afghanistan,Abats Comestible,animale,5.0,1.72,0.77,2017,36296113.0,181480565.0,6.624041e+10,62429314.4,0.00077,10201022.6
1,Afghanistan,"Agrumes, Autres",vegetale,1.0,1.29,0.02,2017,36296113.0,36296113.0,1.324808e+10,46821985.8,0.00002,264961.6
2,Afghanistan,Aliments pour enfants,vegetale,1.0,0.06,0.03,2017,36296113.0,36296113.0,1.324808e+10,2177766.8,0.00003,397442.4
3,Afghanistan,Ananas,vegetale,0.0,0.00,NaN,2017,36296113.0,0.0,0.000000e+00,0.0,NaN,NaN
4,Afghanistan,Bananes,vegetale,4.0,2.70,0.05,2017,36296113.0,145184452.0,5.299232e+10,97999505.1,0.00005,662404.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15600,Îles Salomon,Viande de Suides,animale,45.0,4.70,1.41,2017,636039.0,28621755.0,1.044694e+10,2989383.3,0.00141,327337.5
15601,Îles Salomon,Viande de Volailles,animale,11.0,3.34,1.14,2017,636039.0,6996429.0,2.553697e+09,2124370.3,0.00114,264655.8
15602,Îles Salomon,"Viande, Autre",animale,0.0,0.06,0.04,2017,636039.0,0.0,0.000000e+00,38162.3,0.00004,9286.2
15603,Îles Salomon,Vin,vegetale,0.0,0.07,NaN,2017,636039.0,0.0,0.000000e+00,44522.7,NaN,NaN


In [52]:
# Clarification de la table pour calculer le ratio énergie/poids

ratio_energie = ratio_population.drop(columns = [
    'Disponibilité alimentaire (Kcal/personne/jour)',
    'Disponibilité alimentaire en quantité (kg/personne/an)',
    'Disponibilité de protéines en quantité (g/personne/jour)', 'Population',
    'dispo Kcal/pers/an',
    'Disponibilité de protéines en quantité (KG/personne/jour)'] )
ratio_energie

,Zone,Produit,Origine,Année,dispo Kcal/total/an,dispo kg/total/an,dispo prot_total/kg/an
0,Afghanistan,Abats Comestible,animale,2017,6.624041e+10,62429314.4,10201022.6
1,Afghanistan,"Agrumes, Autres",vegetale,2017,1.324808e+10,46821985.8,264961.6
2,Afghanistan,Aliments pour enfants,vegetale,2017,1.324808e+10,2177766.8,397442.4
3,Afghanistan,Ananas,vegetale,2017,0.000000e+00,0.0,NaN
4,Afghanistan,Bananes,vegetale,2017,5.299232e+10,97999505.1,662404.1
...,...,...,...,...,...,...,...
15600,Îles Salomon,Viande de Suides,animale,2017,1.044694e+10,2989383.3,327337.5
15601,Îles Salomon,Viande de Volailles,animale,2017,2.553697e+09,2124370.3,264655.8
15602,Îles Salomon,"Viande, Autre",animale,2017,0.000000e+00,38162.3,9286.2
15603,Îles Salomon,Vin,vegetale,2017,0.000000e+00,44522.7,NaN


In [53]:
# Correction des erreur et des valeurs = 0 pour ne pas fausser la moyenne. Permettra ensuite de les supprimer plus simplement

ratio_energie.loc[ratio_energie['dispo Kcal/total/an'] == 0, 'dispo Kcal/total/an'] = np.nan
ratio_energie.loc[ratio_energie['dispo Kcal/total/an'] == -21, 'dispo Kcal/total/an'] = np.nan
ratio_energie.loc[ratio_energie['dispo kg/total/an'] == 0, 'dispo kg/total/an'] = np.nan

In [54]:
# Vérification et suppression des NaN

print(ratio_energie.isna().sum())

ratio_energie.dropna(axis='rows', inplace=True)
ratio_energie.isna().sum()

Zone                         0
Produit                      0
Origine                      0
Année                        0
dispo Kcal/total/an       5527
dispo kg/total/an         3594
dispo prot_total/kg/an    4044
dtype: int64


Zone                      0
Produit                   0
Origine                   0
Année                     0
dispo Kcal/total/an       0
dispo kg/total/an         0
dispo prot_total/kg/an    0
dtype: int64

In [55]:
# Calcul du ratio poids/énergie en Kcal

ratio_energie['ratio_energie_poids'] = ratio_energie['dispo Kcal/total/an'] / ratio_energie['dispo kg/total/an']
ratio_energie

,Zone,Produit,Origine,Année,dispo Kcal/total/an,dispo kg/total/an,dispo prot_total/kg/an,ratio_energie_poids
0,Afghanistan,Abats Comestible,animale,2017,6.624041e+10,62429314.4,10201022.6,1061.046511
1,Afghanistan,"Agrumes, Autres",vegetale,2017,1.324808e+10,46821985.8,264961.6,282.945736
2,Afghanistan,Aliments pour enfants,vegetale,2017,1.324808e+10,2177766.8,397442.4,6083.333277
4,Afghanistan,Bananes,vegetale,2017,5.299232e+10,97999505.1,662404.1,540.740741
5,Afghanistan,"Beurre, Ghee",animale,2017,3.047059e+11,42466452.2,397442.4,7175.213677
...,...,...,...,...,...,...,...,...
15598,Îles Salomon,Viande d'Ovins/Caprins,animale,2017,2.321542e+08,89045.5,11607.7,2607.141686
15599,Îles Salomon,Viande de Bovins,animale,2017,6.268164e+09,3542737.2,608244.1,1769.299835
15600,Îles Salomon,Viande de Suides,animale,2017,1.044694e+10,2989383.3,327337.5,3494.680851
15601,Îles Salomon,Viande de Volailles,animale,2017,2.553697e+09,2124370.3,264655.8,1202.095786


### 4.2 Méthode pour obtenir la valeur énergétique des produits

In [56]:
# Exemples pour trouver la valeur énergétique de différents produits (prendre la valeur moyenne)

print(ratio_energie[(ratio_energie['Produit'] == 'Bananes')]['ratio_energie_poids'].describe())
print(ratio_energie[(ratio_energie['Produit'] == 'Patates douces')]['ratio_energie_poids'].describe())

print("Le ratio énergie/poids de la banane est de 617 Kcal (cal/kg), celui de la patate douce est de 967 Kcal")

count     158.000000
mean      617.017036
std        74.550756
min       265.261628
25%       596.544576
50%       611.250698
75%       628.304325
max      1177.419331
Name: ratio_energie_poids, dtype: float64
count     100.000000
mean      967.507766
std       176.252868
min       667.682927
25%       876.900604
50%       953.865705
75%       989.051650
max      1738.095238
Name: ratio_energie_poids, dtype: float64
Le ratio énergie/poids de la banane est de 617 Kcal (cal/kg), celui de la patate douce est de 967 Kcal


In [57]:
# Liste des valeurs énergétiques de tous les produits

ratio_energie.groupby('Produit')['ratio_energie_poids'].mean()

Produit
Abats Comestible            1132.473776
Agrumes, Autres              260.147068
Aliments pour enfants       3826.952015
Ananas                       439.090511
Animaux Aquatiques Autre     618.224061
                               ...     
Viande de Suides            2522.656623
Viande de Volailles         1301.168956
Viande, Autre               1372.485580
Vin                          739.489175
Épices, Autres              3587.285813
Name: ratio_energie_poids, Length: 93, dtype: float64

In [58]:
# Classement des 20 aliments les plus riches

produits_riches = ratio_energie.groupby('Produit')['ratio_energie_poids'].mean().sort_values(ascending=False)
produits_riches.head(20)

Produit
Huiles de Foie de Poisso    11809.004845
Huile de Sésame              9559.523726
Huile de Germe de Maïs       8950.891180
Huiles de Poissons           8917.431214
Huile d'Olive                8774.038462
Huile de Tournesol           8769.634354
Huil Plantes Oleif Autr      8754.156509
Huile de Soja                8737.921304
Huile de Palme               8682.340537
Huile de Colza&Moutarde      8548.729434
Huile Graines de Coton       8478.967022
Huile de Palmistes           8350.682519
Huile de Coco                8288.877047
Huile de Son de Riz          8264.150950
Huile d'Arachide             8237.817984
Graisses Animales Crue       7562.751782
Beurre, Ghee                 7507.566423
Sésame                       6136.984323
Arachides Decortiquees       5705.476239
Palmistes                    5475.000000
Name: ratio_energie_poids, dtype: float64

In [59]:
# Déterminons si la valeur énergétique des produits est plus élevée quand son origine est végétale ou animale : 

ratio_energie.groupby('Origine')['ratio_energie_poids'].mean()

Origine
animale     2085.624181
vegetale    2411.502492
Name: ratio_energie_poids, dtype: float64

In [60]:
# Widget pour visualiser le ratio énergie/poids de chaque produit en fonction de leur disponibilité dans les pays

countries = sorted(list(ratio_energie['Zone'].unique()))
products = sorted(list(ratio_energie['Produit'].unique()))

@interact
def show_per_country(country=countries, product=products):
    return ratio_energie.loc[(ratio_energie['Zone'] == country) & (ratio_energie['Produit'] == product)][['Zone', 'Produit', 'ratio_energie_poids']]

interactive(children=(Dropdown(description='country', options=('Afghanistan', 'Afrique du Sud', 'Albanie', 'Al…

## 5. Utilisation de la disponibilité intérieure

La disponibilité intérieure correspond à la disponibilité totale d’un aliment au sein du pays. Elle se calcule des manières suivantes :
( Production + Importation + Variation des stocks ) – Exportation = DISPONIBILITE INTERIEURE
= Nourriture + Aliments pour animaux + Semences + Autres utilisation + Traitement + Semences + Pertes 

In [61]:
# Clarification de la table

df_dispo_total = dispo_alimentaire.drop(columns = [
    "Exportations - Quantité", 
    "Importations - Quantité", 
    "Production",
    "Semences",
    "Traitement",
    "Variation de stock",
    "Disponibilité alimentaire (Kcal/personne/jour)",
    "Disponibilité alimentaire en quantité (kg/personne/an)", 
    "Disponibilité de matière grasse en quantité (g/personne/jour)",
    "Disponibilité de protéines en quantité (g/personne/jour)"])
df_dispo_total

,Zone,Produit,Origine,Aliments pour animaux,Autres Utilisations,Disponibilité intérieure,Nourriture,Pertes
0,Afghanistan,Abats Comestible,animale,NaN,NaN,53.0,53.0,NaN
1,Afghanistan,"Agrumes, Autres",vegetale,NaN,NaN,41.0,39.0,2.0
2,Afghanistan,Aliments pour enfants,vegetale,NaN,NaN,2.0,2.0,NaN
3,Afghanistan,Ananas,vegetale,NaN,NaN,0.0,0.0,NaN
4,Afghanistan,Bananes,vegetale,NaN,NaN,82.0,82.0,NaN
...,...,...,...,...,...,...,...,...
15600,Îles Salomon,Viande de Suides,animale,NaN,NaN,3.0,3.0,NaN
15601,Îles Salomon,Viande de Volailles,animale,NaN,NaN,2.0,2.0,NaN
15602,Îles Salomon,"Viande, Autre",animale,NaN,NaN,0.0,0.0,NaN
15603,Îles Salomon,Vin,vegetale,NaN,NaN,0.0,0.0,NaN


In [62]:
# Calcul des proportions des différentes utilisations de la disponibilité intérieure

proportion = round((df_dispo_total['Aliments pour animaux'] / df_dispo_total['Disponibilité intérieure']) * 100, 1)
proportion2 = round((df_dispo_total['Autres Utilisations'] / df_dispo_total['Disponibilité intérieure']) * 100, 1)
proportion3 = round((df_dispo_total['Pertes'] / df_dispo_total['Disponibilité intérieure']) * 100, 1)
proportion4 = round((df_dispo_total['Nourriture'] / df_dispo_total['Disponibilité intérieure']) * 100, 1)
df_dispo_total[' % utilisation animale'], df_dispo_total[' % autres utilisations'], df_dispo_total ['% pertes'], df_dispo_total['% nourriture'] = [proportion, proportion2, proportion3, proportion4]
df_dispo_total

,Zone,Produit,Origine,Aliments pour animaux,Autres Utilisations,Disponibilité intérieure,Nourriture,Pertes,% utilisation animale,% autres utilisations,% pertes,% nourriture
0,Afghanistan,Abats Comestible,animale,NaN,NaN,53.0,53.0,NaN,NaN,NaN,NaN,100.0
1,Afghanistan,"Agrumes, Autres",vegetale,NaN,NaN,41.0,39.0,2.0,NaN,NaN,4.9,95.1
2,Afghanistan,Aliments pour enfants,vegetale,NaN,NaN,2.0,2.0,NaN,NaN,NaN,NaN,100.0
3,Afghanistan,Ananas,vegetale,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,Bananes,vegetale,NaN,NaN,82.0,82.0,NaN,NaN,NaN,NaN,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...
15600,Îles Salomon,Viande de Suides,animale,NaN,NaN,3.0,3.0,NaN,NaN,NaN,NaN,100.0
15601,Îles Salomon,Viande de Volailles,animale,NaN,NaN,2.0,2.0,NaN,NaN,NaN,NaN,100.0
15602,Îles Salomon,"Viande, Autre",animale,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN
15603,Îles Salomon,Vin,vegetale,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN


In [63]:
# % de la disponibilité intérieure mondiale utilisée pour nourrir la population 

dispo = df_dispo_total["Disponibilité intérieure"].sum()
nourriture = df_dispo_total["Nourriture"].sum()

proportion_nourriture = round((nourriture/dispo) * 100,1)
print(proportion_nourriture, "% de la disponbilité intérieure mondiale est utilisée pour nourrir les Hommes.")

49.5 % de la disponbilité intérieure mondiale est utilisée pour nourrir les Hommes.


In [64]:
# % de la disponibilité intérieure mondiale utilisée pour nourrir les animaux

animaux = df_dispo_total["Aliments pour animaux"].sum()
proportion_animaux = round((animaux/dispo) * 100,1)

print(proportion_animaux,"% de la disponbilité intérieure mondiale est utilisée pour l'alimentation animale.")

13.2 % de la disponbilité intérieure mondiale est utilisée pour l'alimentation animale.


In [65]:
# % de la disponibilité intérieure mondiale perdue

perte = df_dispo_total["Pertes"].sum()
proportion_perte = round((perte/dispo) * 100,1)

print(proportion_perte,"% de la disponibilité intérieure mondiale est perdue chaque année")

4.6 % de la disponibilité intérieure mondiale est perdue chaque année


In [66]:
# Df spécifique au % de pertes par rapport à la disponibilité intérieure

df_dispo_perte = dispo_alimentaire.drop(columns = [
    "Exportations - Quantité", 
    "Importations - Quantité", 
    "Production",
    "Semences",
    "Traitement",
    "Variation de stock",
    "Disponibilité alimentaire (Kcal/personne/jour)",
    "Disponibilité alimentaire en quantité (kg/personne/an)", 
    "Disponibilité de matière grasse en quantité (g/personne/jour)",
    "Disponibilité de protéines en quantité (g/personne/jour)", 
    "Origine",
    "Aliments pour animaux", 
    "Autres Utilisations", 
    "Nourriture"])
df_dispo_perte['Proportion Pertes'] = proportion3
df_dispo_perte.dropna()

,Zone,Produit,Disponibilité intérieure,Pertes,Proportion Pertes
1,Afghanistan,"Agrumes, Autres",41.0,2.0,4.9
7,Afghanistan,Blé,5992.0,775.0,12.9
16,Afghanistan,"Fruits, Autres",240.0,17.0,7.1
29,Afghanistan,Lait - Excl Beurre,2083.0,58.0,2.8
30,Afghanistan,"Légumes, Autres",1007.0,122.0,12.1
...,...,...,...,...,...
15567,Îles Salomon,Manioc,3.0,0.0,0.0
15572,Îles Salomon,Oeufs,1.0,0.0,0.0
15578,Îles Salomon,Patates douces,100.0,2.0,2.0
15589,Îles Salomon,Racines nda,43.0,1.0,2.3


In [67]:
# Df spécifique au % d'utilisation de la disponibilité intérieure pour nourrir les animaux 

df_utilisation_animale = dispo_alimentaire.drop(columns = ["Exportations - Quantité", "Importations - Quantité", "Production","Semences","Traitement","Variation de stock","Disponibilité alimentaire (Kcal/personne/jour)","Disponibilité alimentaire en quantité (kg/personne/an)", "Disponibilité de matière grasse en quantité (g/personne/jour)","Disponibilité de protéines en quantité (g/personne/jour)", "Origine", "Autres Utilisations", "Nourriture", "Pertes"])
df_utilisation_animale['% utilisation animale'] = proportion
df_utilisation_animale.dropna()

,Zone,Produit,Aliments pour animaux,Disponibilité intérieure,% utilisation animale
29,Afghanistan,Lait - Excl Beurre,123.0,2083.0,5.9
31,Afghanistan,Légumineuses Autres,4.0,86.0,4.7
32,Afghanistan,Maïs,200.0,313.0,63.9
40,Afghanistan,Orge,360.0,524.0,68.7
49,Afghanistan,"Sucre, betterave",0.0,15.0,0.0
...,...,...,...,...,...
15500,Éthiopie,"Plantes Oleiferes, Autre",0.0,279.0,0.0
15504,Éthiopie,Poissons Pelagiques,0.0,1.0,0.0
15564,Îles Salomon,Lait - Excl Beurre,0.0,7.0,0.0
15567,Îles Salomon,Manioc,0.0,3.0,0.0


In [68]:
# Df spécifique au % d'utilisation de la disponibilité intérieure pour nourrir les humains

df_nourriture = dispo_alimentaire.drop(columns = ["Exportations - Quantité", "Importations - Quantité", "Production","Semences","Traitement","Variation de stock","Disponibilité alimentaire (Kcal/personne/jour)","Disponibilité alimentaire en quantité (kg/personne/an)", "Disponibilité de matière grasse en quantité (g/personne/jour)","Disponibilité de protéines en quantité (g/personne/jour)", "Origine", "Autres Utilisations", "Aliments pour animaux", "Pertes"])
df_nourriture['% utilisation humaine'] = proportion4
df_nourriture.dropna()

,Zone,Produit,Disponibilité intérieure,Nourriture,% utilisation humaine
0,Afghanistan,Abats Comestible,53.0,53.0,100.0
1,Afghanistan,"Agrumes, Autres",41.0,39.0,95.1
2,Afghanistan,Aliments pour enfants,2.0,2.0,100.0
4,Afghanistan,Bananes,82.0,82.0,100.0
5,Afghanistan,"Beurre, Ghee",36.0,36.0,100.0
...,...,...,...,...,...
15591,Îles Salomon,Riz (Eq Blanchi),49.0,36.0,73.5
15594,Îles Salomon,Sucre Eq Brut,2.0,2.0,100.0
15599,Îles Salomon,Viande de Bovins,3.0,3.0,100.0
15600,Îles Salomon,Viande de Suides,3.0,3.0,100.0


### 5.2 Le cas du manioc en Thaïlande

La Thaïlande est le 1er pays exportateur mondial de manioc. Elle a produit en 2017 30 millions de tonnes de manioc et près de 50% de sa production annuelle est utlisée pour extraire de l'amidon.

In [69]:
# Localisation des disponbilités du manioc en Thaïlande

dispo_manioc = dispo_alimentaire.loc[(dispo_alimentaire["Zone"] == "Thaïlande") & (dispo_alimentaire["Produit"] == "Manioc")]
dispo_manioc

,Zone,Produit,Origine,Aliments pour animaux,Autres Utilisations,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Semences,Traitement,Variation de stock
13809,Thaïlande,Manioc,vegetale,1800.0,2081.0,40.0,13.0,0.05,0.14,6264.0,25214.0,1250.0,871.0,1511.0,30228.0,NaN,0.0,0.0


In [70]:
# Calcul de la proportion de manioc exportée sur la production totale

exportation_manioc = round((dispo_manioc['Exportations - Quantité'] / dispo_manioc['Production']) * 100, 1 )
print(exportation_manioc)

13809    83.4
dtype: float64


In [71]:
print("La Thaïlande exporte 83.4% de sa production annuelle de manioc")

La Thaïlande exporte 83.4% de sa production annuelle de manioc


In [72]:
nourriture_manioc = round((dispo_manioc['Nourriture'] / dispo_manioc['Production']) * 100, 1 )
print(nourriture_manioc)

13809    2.9
dtype: float64


In [73]:
print("Seulement 2.9% de la production de manioc est destinée à nourrir la population du pays")

Seulement 2.9% de la production de manioc est destinée à nourrir la population du pays


In [74]:
# Calories par kg de manioc
kcal_manioc = 1570

# Conversion de la part annuelle de manioc exportée en KG
manioc_kilo = 25214 * 1000000

# Calcul de la disponibilité calorique de la quantité de manioc exportée chaque année
apport_kcal_manioc = kcal_manioc * manioc_kilo

# Calcul du nombre de personne qui pourrait être nourrie chaque année si la quantité de manioc dédiée à l'exportation était utilisée pour nourrir la population
nbr_pers_nourrie_manioc = round((apport_kcal_manioc / 2400) / 365,0)

# Calcul du % de la population de Thaïlande que cela représente
proportion_population = round((45.2/69.2) * 100,1)

print("Chaque année, la Thaïlande exporte", manioc_kilo,"KG de manioc. Si cette part était utilisée pour nourrir sa population, la Thaïlande pourrait subvenir aux besoins en alimentation de", nbr_pers_nourrie_manioc, "individus, cela représente",proportion_population,"% de sa population." )

Chaque année, la Thaïlande exporte 25214000000 KG de manioc. Si cette part était utilisée pour nourrir sa population, la Thaïlande pourrait subvenir aux besoins en alimentation de 45189475.0 individus, cela représente 65.3 % de sa population.


In [75]:
# Recherche des informations nécessaires au calcul : on notera que la valeur énergétique du manioc est de 988 Kcal, chiffre trop éloigné de la réalité. Je continuerai donc d'utiliser sa valeur "normale" de 1570 Kcal. 

print(comparaison.loc[comparaison['Zone'] == "Thaïlande"])
print(ratio_energie[(ratio_energie['Produit'] == 'Manioc')]['ratio_energie_poids'].describe())

          Zone  Annee  Population  population_sous_nutrition  proportion
185  Thaïlande   2017        69.2                        6.2         9.0
count      88.000000
mean      988.017113
std       142.617718
min       579.365087
25%       898.086252
50%      1004.227401
75%      1079.401046
max      1617.411026
Name: ratio_energie_poids, dtype: float64


In [76]:
# Calcul des besoins calorifiques pour nourrir les personnes en état de sous-nutrition pendant un an sur une base de 2400 kcal/j/personnne

besoin_kcal_thailande = (6200000 * 2400) * 365
print(besoin_kcal_thailande)

5431200000000


In [77]:
# Calcul du nombre de KG de manioc nécessaire pour satisfaire aux besoins calorifiques de sa population en état de sous-nutrition

besoin_kg = round(besoin_kcal_thailande / 1570,1)
print(besoin_kg)

3459363057.3


In [78]:
# Calcul de la proportion que cela représente des exportations de manioc 

proportion_besoin = round((besoin_kg / manioc_kilo) * 100,1)
print("Si la Thaïlande réduisait de",proportion_besoin,"% son exportation de manioc et qu'elle utilisait cette part pour nourrir les hommes, elle pourrait régler le problème de sous-nutrition dans le pays.")

Si la Thaïlande réduisait de 13.7 % son exportation de manioc et qu'elle utilisait cette part pour nourrir les hommes, elle pourrait régler le problème de sous-nutrition dans le pays.


## 6. Calcul des aides reçues par pays / année / produit

L'aide alimentaire de projet est généralement fournie pour soutenir des activités spécifiques de réduction de la pauvreté et de prévention des catastrophes, ciblées sur des groupes ou des zones bénéficiaires spécifiques ; l'aide alimentaire de secours est ciblée sur les victimes de catastrophes naturelles ou d'origine humaine et leur est distribuée gratuitement. Toutefois, l'aide alimentaire peut également s'inscrire dans le long terme. Elle peut être utilisée pour développer des solutions durables dans des zones où il existe des pénuries alimentaires et créer une sécurité alimentaire pour aider à briser le cycle de la dépendance à l'aide internationale. 

In [79]:
# Classement des pays ayant bénéficié du plus d'aide depuis 2013

pays_benef = df_aide.groupby('Pays bénéficiaire')[['Valeur']].sum()
pays_benef.sort_values(by = 'Valeur', ascending = False)

,Valeur
Pays bénéficiaire,
République arabe syrienne,1858943
Éthiopie,1381294
Yémen,1206484
Soudan du Sud,695248
Soudan,669784
...,...
Égypte,1122
Vanuatu,802
Timor-Leste,116


In [80]:
# Calcul du % reçu par pays de l'aide totale délivrée depuis 2013

pays_benef['proportion_total_aide'] = (pays_benef['Valeur'] / pays_benef['Valeur'].sum()) * 100
pays_benef.sort_values(by = 'proportion_total_aide', ascending = False).head(10)

,Valeur,proportion_total_aide
Pays bénéficiaire,,
République arabe syrienne,1858943,16.844506
Éthiopie,1381294,12.516368
Yémen,1206484,10.932356
Soudan du Sud,695248,6.299875
Soudan,669784,6.069137
Kenya,552836,5.009432
Bangladesh,348188,3.155048
Somalie,292678,2.652054
République démocratique du Congo,288502,2.614213


In [81]:
# Total de l'aide fournie chaque année en tonnes de produits

df_aide.groupby('Année')[['Valeur']].sum()

,Valeur
Année,
2013,4165674
2014,3939152
2015,2187507
2016,743568


In [82]:
# Tableau de l'aide fournie pour chaque produit par année

table_aide_produit = pd.pivot_table(data = df_aide,index = ['Produit'], columns = ['Année'], aggfunc = sum) 
table_aide_produit["total_aide (tonnes)"] = table_aide_produit.sum(axis=1)

print("Les céréales sont de très loin l'aide alimentaire la plus distribuée chaque année :")
table_aide_produit.sort_values(by = 'total_aide (tonnes)', ascending = False)

Les céréales sont de très loin l'aide alimentaire la plus distribuée chaque année :


Valeur                                 \
Année                          2013       2014      2015      2016   
Produit                                                              
Céréales                  1589646.0  1571543.0  810346.0  285349.0   
Blé et Farin               656791.0   559060.0  374904.0  190760.0   
Non-céréales               493304.0   398314.0  283460.0   86435.0   
Céréales Secondaires       448191.0   598961.0  118235.0    1178.0   
Riz, total                 307075.0   217823.0  162746.0   54774.0   
Légumineuses Sèches,Tot.   286741.0   253681.0  146697.0   50279.0   
Mélanges et préparations   144948.0   127981.0  126398.0   26512.0   
Huiles végétales           151443.0   107539.0   71766.0   14225.0   
BulgurFarEnt                32641.0    67718.0   28063.0   12125.0   
Sucre, total                31639.0    13189.0   44718.0   11468.0   
Autres non-céréales         13359.0    15211.0   12729.0    7572.0   
Lait, total                  3763.0     5690.0    2250.0     775.0   
Fruits secs, total           3410.0      540.0    3392.0    2044.0   
Poiss&produi                 2672.0     1850.0    1803.0      72.0   
Viande Total                    1.0       52.0       NaN       NaN   
Graisses Com                   50.0        NaN       NaN       NaN   

                         total_aide (tonnes)  
Année                                         
Produit                                       
Céréales                           4256884.0  
Blé et Farin                       1781515.0  
Non-céréales                       1261513.0  
Céréales Secondaires               1166565.0  
Riz, total                          742418.0  
Légumineuses Sèches,Tot.            737398.0  
Mélanges et préparations            425839.0  
Huiles végétales                    344973.0  
BulgurFarEnt                        140547.0  
Sucre, total                        101014.0  
Autres non-céréales                  48871.0  
Lait, total                          12478.0  
Fruits secs, total                    9386.0  
Poiss&produi                          6397.0  
Viande Total                            53.0  
Graisses Com                            50.0

In [83]:
total_aide = table_aide_produit["total_aide (tonnes)"].sum()
total_aide

11035901.0

In [84]:
# Calcul du % d'aide distribuée en céréales

proportion_cereales = (4256884 / total_aide) * 100
proportion_cereales

38.57305352775455

In [85]:
# Tableau de l'aide fournie pour chaque pays par année

table_aide = pd.pivot_table(data = df_aide,index = ['Pays bénéficiaire'], columns = ['Année'], aggfunc = sum) 
table_aide["total_aide"] = table_aide.sum(axis=1)

print("Les 5 pays ayant reçu le plus d'aide depuis 2013 sont : République arabe syrienne, l'Ethiopie, le Yémen, le Soundan du sud et le Soudan")
table_aide.sort_values(by = 'total_aide', ascending = False).head(10)

Les 5 pays ayant reçu le plus d'aide depuis 2013 sont : République arabe syrienne, l'Ethiopie, le Yémen, le Soundan du sud et le Soudan


Valeur                                \
Année                                 2013      2014      2015      2016   
Pays bénéficiaire                                                          
République arabe syrienne         563566.0  651870.0  524949.0  118558.0   
Éthiopie                          591404.0  586624.0  203266.0       NaN   
Yémen                             264764.0  103840.0  372306.0  465574.0   
Soudan du Sud                     196330.0  450610.0   48308.0       NaN   
Soudan                            330230.0  321904.0   17650.0       NaN   
Kenya                             220966.0  217418.0  114452.0       NaN   
Bangladesh                        131018.0  194628.0   22542.0       NaN   
Somalie                           139800.0   81180.0   71698.0       NaN   
République démocratique du Congo  150320.0   70134.0   68048.0       NaN   
Niger                              62720.0   66226.0   54656.0   92742.0   

                                 total_aide  
Année                                        
Pays bénéficiaire                            
République arabe syrienne         1858943.0  
Éthiopie                          1381294.0  
Yémen                             1206484.0  
Soudan du Sud                      695248.0  
Soudan                             669784.0  
Kenya                              552836.0  
Bangladesh                         348188.0  
Somalie                            292678.0  
République démocratique du Congo   288502.0  
Niger                              276344.0

## 6. Focus sur les céréales

Aujourd'hui, les céréales fournissent plus d'énergie dans le monde que toute autre source alimentaire et ont joué un rôle central dans la campagne Néanmoins, bien que très abordables, il est nécessaire de garder en tête  ces céréales ne fournissent pas suffisamment de nutriments nécessaires à une bonne santé, comme le fer, le zinc ou la vitamine A, pour en faire un régime alimentaire adapté.

### 6.1 Utilisation des céréales confondues

In [86]:
# Copie dédiée aux céréales d'un df existant

df_cereales = dispo_alimentaire.copy()

In [87]:
# Sélection des produits céréaliers (selon le FAO)

df_cereales = df_cereales.loc[(df_cereales["Produit"] == "Millet") | (df_cereales["Produit"] == "Orge") | (df_cereales["Produit"] == "Seigle")| (df_cereales["Produit"] == "Céréales, Autres")| (df_cereales["Produit"] == "Blé")| (df_cereales["Produit"] == "Maïs")| (df_cereales["Produit"] == "Avoine") | (df_cereales["Produit"] == "Riz (Eq Blanchi)") | (df_cereales["Produit"] == "Sorgho")]
df_cereales

# Une autre alternative serait  de créer une liste : list_cereal = ['Maïs','Millet','Orge','Blé','Céréales, Autres','Sorgho','Riz (Eq Blanchi)','Seigle','Avoine'] df_cereales.loc[(df_cereales["Produit"] .isin(list_cereal),:]

,Zone,Produit,Origine,Aliments pour animaux,Autres Utilisations,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Semences,Traitement,Variation de stock
7,Afghanistan,Blé,vegetale,NaN,NaN,1369.0,160.23,4.69,36.91,5992.0,NaN,1173.0,4895.0,775.0,5169.0,322.0,NaN,-350.0
12,Afghanistan,"Céréales, Autres",vegetale,NaN,NaN,0.0,0.00,0.00,0.00,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN
32,Afghanistan,Maïs,vegetale,200.0,NaN,21.0,2.50,0.30,0.56,313.0,0.0,1.0,76.0,31.0,312.0,5.0,NaN,NaN
34,Afghanistan,Millet,vegetale,NaN,NaN,3.0,0.40,0.02,0.08,13.0,NaN,NaN,12.0,1.0,13.0,0.0,NaN,NaN
40,Afghanistan,Orge,vegetale,360.0,NaN,26.0,2.92,0.24,0.79,524.0,NaN,10.0,89.0,52.0,514.0,22.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15545,Îles Salomon,"Céréales, Autres",vegetale,NaN,NaN,0.0,0.00,0.00,0.00,0.0,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN
15568,Îles Salomon,Maïs,vegetale,NaN,NaN,1.0,0.15,0.01,0.03,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN
15575,Îles Salomon,Orge,vegetale,NaN,NaN,0.0,0.07,NaN,0.01,1.0,NaN,1.0,0.0,NaN,NaN,NaN,1.0,NaN
15591,Îles Salomon,Riz (Eq Blanchi),vegetale,0.0,12.0,623.0,63.76,1.36,10.90,49.0,0.0,47.0,36.0,1.0,3.0,0.0,NaN,0.0


In [88]:
# Aggregation par pays

table_cereales = pd.pivot_table(data=df_cereales,index=['Zone'],aggfunc="sum")
table_cereales

,Aliments pour animaux,Autres Utilisations,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Semences,Traitement,Variation de stock
Zone,,,,,,,,,,,,,,,
Afghanistan,560.0,0.0,1560.0,179.87,5.52,41.04,7303.0,0.0,1303.0,5494.0,883.0,6350.0,363.0,0.0,-350.0
Afrique du Sud,4824.0,0.0,1538.0,180.02,10.20,41.09,15493.0,3777.0,3147.0,9503.0,500.0,14155.0,73.0,593.0,1969.0
Albanie,411.0,134.0,1144.0,149.27,6.08,35.49,1145.0,4.0,468.0,474.0,91.0,703.0,22.0,12.0,-22.0
Algérie,4337.0,820.0,1678.0,217.67,6.79,49.33,15013.0,2.0,10279.0,8535.0,1024.0,4914.0,253.0,42.0,-180.0
Allemagne,26615.0,1013.0,872.0,111.11,3.48,26.39,42158.0,16721.0,11093.0,9191.0,1221.0,47757.0,1027.0,3090.0,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Émirats arabes unis,1053.0,263.0,1379.0,158.68,5.66,34.21,3142.0,619.0,4249.0,1484.0,384.0,55.0,0.0,0.0,-543.0
Équateur,847.0,787.0,812.0,95.57,3.12,18.85,3407.0,53.0,877.0,1505.0,138.0,2598.0,39.0,93.0,-15.0
États-Unis d'Amérique,140096.0,137181.0,801.0,105.63,3.63,23.70,343006.0,66302.0,13477.0,33810.0,222.0,433686.0,3098.0,28608.0,-37854.0


In [89]:
# Quelques calculs sur l'utilisation des céréales

dispo_cereale = table_cereales['Disponibilité intérieure'].sum()
utilisation_animale = table_cereales['Aliments pour animaux'].sum()
exportation = table_cereales['Exportations - Quantité'].sum()
nourriture = table_cereales['Nourriture'].sum()
perte = table_cereales['Pertes'].sum()

proportion = round((utilisation_animale/dispo_cereale) * 100,1)
print("Proportion dédiée à nourrir les animaux :", proportion,"%")

proportion_exportation = round((exportation/dispo_cereale) * 100,1)
print("Proportion exportée:",proportion_exportation,"%")

proportion_nourriture = round((nourriture/dispo_cereale) * 100,1)
print("Proportion utilisée pour nourrir les Hommes:",proportion_nourriture,"%")

proportion_perte = round((perte/dispo_cereale) * 100,1)
print("Proportion perdue:",proportion_perte,"%")

Proportion dédiée à nourrir les animaux : 36.3 %
Proportion exportée: 17.7 %
Proportion utilisée pour nourrir les Hommes: 42.8 %
Proportion perdue: 4.5 %


In [90]:
# Classement des 10 plus gros producteurs toutes céréales confondues

table_cereales.nlargest(10, ['Production'])

,Aliments pour animaux,Autres Utilisations,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Semences,Traitement,Variation de stock
Zone,,,,,,,,,,,,,,,
"Chine, continentale",193042.0,40180.0,1428.0,150.89,5.97,33.86,483482.0,2084.0,14786.0,209038.0,20199.0,485073.0,11173.0,9848.0,-14294.0
États-Unis d'Amérique,140096.0,137181.0,801.0,105.63,3.63,23.70,343006.0,66302.0,13477.0,33810.0,222.0,433686.0,3098.0,28608.0,-37854.0
Inde,11419.0,45.0,1361.0,148.46,5.68,32.33,216517.0,24546.0,114.0,185886.0,12059.0,240926.0,6703.0,470.0,23.0
Brésil,43264.0,1.0,969.0,116.24,2.88,22.66,79455.0,28922.0,10768.0,23290.0,9775.0,97157.0,780.0,2346.0,451.0
Fédération de Russie,30806.0,19.0,1153.0,148.00,3.93,33.81,68227.0,19557.0,2298.0,21141.0,974.0,90073.0,11212.0,4077.0,-4584.0
France,19103.0,3437.0,960.0,127.24,4.67,29.33,35536.0,36893.0,3726.0,8181.0,575.0,67469.0,1169.0,3073.0,1231.0
Canada,23905.0,950.0,909.0,119.37,3.29,26.34,32769.0,27245.0,2780.0,4200.0,492.0,66406.0,1361.0,2198.0,-9169.0
Indonésie,8045.0,11956.0,1751.0,195.48,6.92,35.39,74532.0,339.0,10955.0,48845.0,5077.0,66056.0,584.0,66.0,-2141.0
Ukraine,19890.0,36.0,1060.0,138.41,3.85,30.80,32235.0,27789.0,275.0,6261.0,1947.0,62638.0,2634.0,1467.0,-2890.0


In [91]:
# Classement des 10 plus gros exportateurs toutes céréales confondues

table_cereales.nlargest(10, ['Exportations - Quantité'])

,Aliments pour animaux,Autres Utilisations,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Semences,Traitement,Variation de stock
Zone,,,,,,,,,,,,,,,
États-Unis d'Amérique,140096.0,137181.0,801.0,105.63,3.63,23.70,343006.0,66302.0,13477.0,33810.0,222.0,433686.0,3098.0,28608.0,-37854.0
France,19103.0,3437.0,960.0,127.24,4.67,29.33,35536.0,36893.0,3726.0,8181.0,575.0,67469.0,1169.0,3073.0,1231.0
Argentine,6533.0,1285.0,1060.0,123.43,2.99,27.38,16176.0,29821.0,22.0,5116.0,1266.0,51398.0,1132.0,844.0,-5424.0
Brésil,43264.0,1.0,969.0,116.24,2.88,22.66,79455.0,28922.0,10768.0,23290.0,9775.0,97157.0,780.0,2346.0,451.0
Ukraine,19890.0,36.0,1060.0,138.41,3.85,30.80,32235.0,27789.0,275.0,6261.0,1947.0,62638.0,2634.0,1467.0,-2890.0
Canada,23905.0,950.0,909.0,119.37,3.29,26.34,32769.0,27245.0,2780.0,4200.0,492.0,66406.0,1361.0,2198.0,-9169.0
Australie,7936.0,1046.0,741.0,87.71,2.84,21.70,12608.0,25951.0,522.0,2048.0,307.0,35212.0,856.0,415.0,2827.0
Inde,11419.0,45.0,1361.0,148.46,5.68,32.33,216517.0,24546.0,114.0,185886.0,12059.0,240926.0,6703.0,470.0,23.0
Fédération de Russie,30806.0,19.0,1153.0,148.00,3.93,33.81,68227.0,19557.0,2298.0,21141.0,974.0,90073.0,11212.0,4077.0,-4584.0


In [92]:
# Calcul de l'utilisation de la disponibilité intérieure céréalière par pays

cereales_animaux = round((table_cereales['Aliments pour animaux']/ table_cereales['Disponibilité intérieure']) * 100,2)
table_cereales['% utilisation pour animaux'] = cereales_animaux

utilisation_humaine = round((table_cereales['Nourriture']/ table_cereales['Disponibilité intérieure']) * 100,2)
table_cereales['% utilisation nourriture humaine'] = utilisation_humaine

table_cereales

,Aliments pour animaux,Autres Utilisations,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Semences,Traitement,Variation de stock,% utilisation pour animaux,% utilisation nourriture humaine
Zone,,,,,,,,,,,,,,,,,
Afghanistan,560.0,0.0,1560.0,179.87,5.52,41.04,7303.0,0.0,1303.0,5494.0,883.0,6350.0,363.0,0.0,-350.0,7.67,75.23
Afrique du Sud,4824.0,0.0,1538.0,180.02,10.20,41.09,15493.0,3777.0,3147.0,9503.0,500.0,14155.0,73.0,593.0,1969.0,31.14,61.34
Albanie,411.0,134.0,1144.0,149.27,6.08,35.49,1145.0,4.0,468.0,474.0,91.0,703.0,22.0,12.0,-22.0,35.90,41.40
Algérie,4337.0,820.0,1678.0,217.67,6.79,49.33,15013.0,2.0,10279.0,8535.0,1024.0,4914.0,253.0,42.0,-180.0,28.89,56.85
Allemagne,26615.0,1013.0,872.0,111.11,3.48,26.39,42158.0,16721.0,11093.0,9191.0,1221.0,47757.0,1027.0,3090.0,28.0,63.13,21.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Émirats arabes unis,1053.0,263.0,1379.0,158.68,5.66,34.21,3142.0,619.0,4249.0,1484.0,384.0,55.0,0.0,0.0,-543.0,33.51,47.23
Équateur,847.0,787.0,812.0,95.57,3.12,18.85,3407.0,53.0,877.0,1505.0,138.0,2598.0,39.0,93.0,-15.0,24.86,44.17
États-Unis d'Amérique,140096.0,137181.0,801.0,105.63,3.63,23.70,343006.0,66302.0,13477.0,33810.0,222.0,433686.0,3098.0,28608.0,-37854.0,40.84,9.86


In [93]:
table_cereales.nlargest(20, ['% utilisation nourriture humaine'])

,Aliments pour animaux,Autres Utilisations,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Semences,Traitement,Variation de stock,% utilisation pour animaux,% utilisation nourriture humaine
Zone,,,,,,,,,,,,,,,,,
Sao Tomé-et-Principe,0.0,0.0,868.0,104.55,4.07,22.18,20.0,0.0,20.0,20.0,0.0,1.0,0.0,0.0,0.0,0.00,100.00
Chine - RAS de Hong-Kong,2.0,22.0,904.0,106.03,2.48,18.78,802.0,111.0,908.0,763.0,3.0,0.0,0.0,8.0,4.0,0.25,95.14
Djibouti,0.0,4.0,1419.0,167.41,5.20,37.22,155.0,1.0,929.0,146.0,5.0,0.0,0.0,0.0,-773.0,0.00,94.19
Timor-Leste,8.0,0.0,1259.0,161.68,8.08,27.22,197.0,0.0,31.0,184.0,5.0,159.0,2.0,0.0,8.0,4.06,93.40
Kiribati,0.0,1.0,1016.0,114.94,2.66,22.67,13.0,0.0,13.0,12.0,0.0,0.0,0.0,0.0,0.0,0.00,92.31
Mauritanie,12.0,0.0,1492.0,172.11,6.72,39.00,724.0,0.0,592.0,668.0,36.0,244.0,7.0,0.0,-113.0,1.66,92.27
Lesotho,1.0,0.0,1934.0,226.01,17.93,52.12,514.0,32.0,315.0,469.0,32.0,120.0,4.0,8.0,110.0,0.19,91.25
Yémen,210.0,4.0,1355.0,161.79,9.77,38.10,4354.0,183.0,4124.0,3950.0,152.0,863.0,33.0,8.0,-449.0,4.82,90.72
Kenya,146.0,83.0,1075.0,128.02,8.69,28.46,6303.0,53.0,1787.0,5677.0,146.0,4259.0,81.0,171.0,309.0,2.32,90.07


### 6.2 Valeur énergétique des céréales

In [94]:
# Création d'un df ratio énergie/poids pour les céréales

ratio_energie_cereales = ratio_energie.loc[(ratio_energie["Produit"] == "Millet") | (ratio_energie["Produit"] == "Orge") | (ratio_energie["Produit"] == "Seigle")| (ratio_energie["Produit"] == "Céréales, Autres")| (ratio_energie["Produit"] == "Blé")| (ratio_energie["Produit"] == "Sorgho")| (ratio_energie["Produit"] == "Maïs")| (ratio_energie["Produit"] == "Avoine") | (ratio_energie["Produit"] == "Riz (Eq Blanchi)")]
ratio_energie_cereales

,Zone,Produit,Origine,Année,dispo Kcal/total/an,dispo kg/total/an,dispo prot_total/kg/an,ratio_energie_poids
7,Afghanistan,Blé,vegetale,2017,1.813662e+13,5.815726e+09,488986678.8,3118.548337
32,Afghanistan,Maïs,vegetale,2017,2.782097e+11,9.074028e+07,7418925.5,3066.000000
34,Afghanistan,Millet,vegetale,2017,3.974424e+10,1.451845e+07,1059846.5,2737.500000
40,Afghanistan,Orge,vegetale,2017,3.444501e+11,1.059846e+08,10465984.2,3249.999999
47,Afghanistan,Riz (Eq Blanchi),vegetale,2017,1.867979e+12,5.016123e+08,35769819.4,3723.950796
...,...,...,...,...,...,...,...,...
15510,Éthiopie,Riz (Eq Blanchi),vegetale,2017,8.543914e+11,2.393998e+08,15534388.9,3568.888889
15513,Éthiopie,Sorgho,vegetale,2017,8.388570e+12,2.714262e+09,246608423.9,3090.552724
15537,Îles Salomon,Blé,vegetale,2017,4.271638e+10,1.613631e+07,1204880.5,2647.221132
15568,Îles Salomon,Maïs,vegetale,2017,2.321542e+08,9.540580e+04,6964.6,2433.334609


In [95]:
# Jointure des 2 tables 

cereale = pd.merge(ratio_energie_cereales, df_cereales, on=["Zone", "Produit"])
cereale

,Zone,Produit,Origine_x,Année,dispo Kcal/total/an,dispo kg/total/an,dispo prot_total/kg/an,ratio_energie_poids,Origine_y,Aliments pour animaux,...,Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Semences,Traitement,Variation de stock
0,Afghanistan,Blé,vegetale,2017,1.813662e+13,5.815726e+09,488986678.8,3118.548337,vegetale,NaN,...,36.91,5992.0,NaN,1173.0,4895.0,775.0,5169.0,322.0,NaN,-350.0
1,Afghanistan,Maïs,vegetale,2017,2.782097e+11,9.074028e+07,7418925.5,3066.000000,vegetale,200.0,...,0.56,313.0,0.0,1.0,76.0,31.0,312.0,5.0,NaN,NaN
2,Afghanistan,Millet,vegetale,2017,3.974424e+10,1.451845e+07,1059846.5,2737.500000,vegetale,NaN,...,0.08,13.0,NaN,NaN,12.0,1.0,13.0,0.0,NaN,NaN
3,Afghanistan,Orge,vegetale,2017,3.444501e+11,1.059846e+08,10465984.2,3249.999999,vegetale,360.0,...,0.79,524.0,NaN,10.0,89.0,52.0,514.0,22.0,NaN,0.0
4,Afghanistan,Riz (Eq Blanchi),vegetale,2017,1.867979e+12,5.016123e+08,35769819.4,3723.950796,vegetale,NaN,...,2.70,461.0,NaN,119.0,422.0,24.0,342.0,14.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
980,Éthiopie,Riz (Eq Blanchi),vegetale,2017,8.543914e+11,2.393998e+08,15534388.9,3568.888889,vegetale,NaN,...,0.40,212.0,0.0,89.0,212.0,NaN,123.0,NaN,0.0,0.0
981,Éthiopie,Sorgho,vegetale,2017,8.388570e+12,2.714262e+09,246608423.9,3090.552724,vegetale,NaN,...,6.35,4156.0,12.0,30.0,2401.0,218.0,4338.0,37.0,NaN,-200.0
982,Îles Salomon,Blé,vegetale,2017,4.271638e+10,1.613631e+07,1204880.5,2647.221132,vegetale,NaN,...,5.19,14.0,0.0,14.0,14.0,NaN,NaN,NaN,NaN,0.0
983,Îles Salomon,Maïs,vegetale,2017,2.321542e+08,9.540580e+04,6964.6,2433.334609,vegetale,NaN,...,0.03,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN


In [96]:
cereale = cereale.drop(columns = ['dispo Kcal/total/an', 'dispo kg/total/an',
       'dispo prot_total/kg/an', 'Origine_y',
       'Disponibilité alimentaire (Kcal/personne/jour)',
       'Disponibilité alimentaire en quantité (kg/personne/an)',
       'Disponibilité de matière grasse en quantité (g/personne/jour)',
       'Disponibilité de protéines en quantité (g/personne/jour)','Semences', 'Traitement', 'Variation de stock'
       ])
cereale

,Zone,Produit,Origine_x,Année,ratio_energie_poids,Aliments pour animaux,Autres Utilisations,Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production
0,Afghanistan,Blé,vegetale,2017,3118.548337,NaN,NaN,5992.0,NaN,1173.0,4895.0,775.0,5169.0
1,Afghanistan,Maïs,vegetale,2017,3066.000000,200.0,NaN,313.0,0.0,1.0,76.0,31.0,312.0
2,Afghanistan,Millet,vegetale,2017,2737.500000,NaN,NaN,13.0,NaN,NaN,12.0,1.0,13.0
3,Afghanistan,Orge,vegetale,2017,3249.999999,360.0,NaN,524.0,NaN,10.0,89.0,52.0,514.0
4,Afghanistan,Riz (Eq Blanchi),vegetale,2017,3723.950796,NaN,NaN,461.0,NaN,119.0,422.0,24.0,342.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
980,Éthiopie,Riz (Eq Blanchi),vegetale,2017,3568.888889,NaN,NaN,212.0,0.0,89.0,212.0,NaN,123.0
981,Éthiopie,Sorgho,vegetale,2017,3090.552724,NaN,1500.0,4156.0,12.0,30.0,2401.0,218.0,4338.0
982,Îles Salomon,Blé,vegetale,2017,2647.221132,NaN,NaN,14.0,0.0,14.0,14.0,NaN,NaN
983,Îles Salomon,Maïs,vegetale,2017,2433.334609,NaN,NaN,0.0,NaN,0.0,0.0,NaN,NaN


In [97]:
# Calcul de l'utilisation de la disponibilité intérieure pour chaque céréale par pays

cereales_animaux = round((cereale['Aliments pour animaux']/ cereale['Disponibilité intérieure']) * 100,2)
cereale['% utilisation pour animaux'] = cereales_animaux

utilisation_humaine = round((cereale['Nourriture']/ cereale['Disponibilité intérieure']) * 100,2)
cereale['% utilisation nourriture humaine'] = utilisation_humaine

cereale

,Zone,Produit,Origine_x,Année,ratio_energie_poids,Aliments pour animaux,Autres Utilisations,Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,% utilisation pour animaux,% utilisation nourriture humaine
0,Afghanistan,Blé,vegetale,2017,3118.548337,NaN,NaN,5992.0,NaN,1173.0,4895.0,775.0,5169.0,NaN,81.69
1,Afghanistan,Maïs,vegetale,2017,3066.000000,200.0,NaN,313.0,0.0,1.0,76.0,31.0,312.0,63.9,24.28
2,Afghanistan,Millet,vegetale,2017,2737.500000,NaN,NaN,13.0,NaN,NaN,12.0,1.0,13.0,NaN,92.31
3,Afghanistan,Orge,vegetale,2017,3249.999999,360.0,NaN,524.0,NaN,10.0,89.0,52.0,514.0,68.7,16.98
4,Afghanistan,Riz (Eq Blanchi),vegetale,2017,3723.950796,NaN,NaN,461.0,NaN,119.0,422.0,24.0,342.0,NaN,91.54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
980,Éthiopie,Riz (Eq Blanchi),vegetale,2017,3568.888889,NaN,NaN,212.0,0.0,89.0,212.0,NaN,123.0,NaN,100.00
981,Éthiopie,Sorgho,vegetale,2017,3090.552724,NaN,1500.0,4156.0,12.0,30.0,2401.0,218.0,4338.0,NaN,57.77
982,Îles Salomon,Blé,vegetale,2017,2647.221132,NaN,NaN,14.0,0.0,14.0,14.0,NaN,NaN,NaN,100.00
983,Îles Salomon,Maïs,vegetale,2017,2433.334609,NaN,NaN,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN


In [98]:
cereale.nlargest(10, ['Exportations - Quantité'])

,Zone,Produit,Origine_x,Année,ratio_energie_poids,Aliments pour animaux,Autres Utilisations,Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,% utilisation pour animaux,% utilisation nourriture humaine
967,États-Unis d'Amérique,Blé,vegetale,2017,2718.326495,6196.0,16.0,34051.0,34691.0,5491.0,25742.0,NaN,57967.0,18.20,75.60
138,Brésil,Maïs,vegetale,2017,3127.454481,40487.0,NaN,54737.0,26780.0,979.0,5612.0,8145.0,80538.0,73.97,10.25
969,États-Unis d'Amérique,Maïs,vegetale,2017,2743.464052,128024.0,137023.0,292776.0,24655.0,3595.0,3917.0,NaN,353699.0,43.73,1.34
314,France,Blé,vegetale,2017,2730.010145,7822.0,2824.0,20298.0,21502.0,2055.0,6971.0,358.0,38614.0,38.54,34.34
181,Canada,Blé,vegetale,2017,2736.640772,3731.0,664.0,8381.0,20704.0,818.0,2989.0,21.0,37530.0,44.52,35.66
51,Argentine,Maïs,vegetale,2017,3006.704981,4923.0,1266.0,7881.0,20241.0,3.0,433.0,516.0,32119.0,62.47,5.49
59,Australie,Blé,vegetale,2017,3025.262560,3200.0,920.0,6691.0,18171.0,283.0,1645.0,229.0,22856.0,47.83,24.59
915,Ukraine,Maïs,vegetale,2017,2717.719133,11393.0,14.0,14190.0,16816.0,57.0,480.0,1235.0,30950.0,80.29,3.38
321,Fédération de Russie,Blé,vegetale,2017,2847.774639,8142.0,3.0,34708.0,14243.0,1360.0,18710.0,399.0,52091.0,23.46,53.91
410,Inde,Riz (Eq Blanchi),vegetale,2017,3619.009929,2124.0,NaN,94854.0,11337.0,5.0,87006.0,3186.0,106186.0,2.24,91.73


### 6.2.a Focus sur l'utilisation des céréales aux USA

In [99]:
print(ratio_energie[(ratio_energie['Produit'] == 'Blé')]['ratio_energie_poids'].describe())

count     174.000000
mean     2851.016900
std       212.546643
min      2308.891661
25%      2716.573492
50%      2830.089994
75%      2941.114877
max      3810.951188
Name: ratio_energie_poids, dtype: float64


In [100]:
# Calories par kg de blé
kcal_blé = 2830

# Conversion de la part annuelle de blé exportée en KG par les USA
blé_kilo = 34691 * 1000000

# Calcul de la disponibilité calorique de la quantité de blé exportée chaque année
apport_kcal_blé = kcal_blé * blé_kilo

# Calcul du nombre de personne qui pourrait être nourrie chaque année si la quantité de blé dédiée à l'exportation était utilisée pour nourrir la population
nbr_pers_nourrie_blé = round((apport_kcal_blé / 2400) / 365,0)

print("Chaque année, les USA exportent", blé_kilo,"KG de blé. Si cette part était utilisée pour nourrir la population mondiale, les USA pourraient subvenir aux besoins en alimentation de", nbr_pers_nourrie_blé, "individus par an, soit 1/5 de la population en état de sous-nutrition." )

Chaque année, les USA exportent 34691000000 KG de blé. Si cette part était utilisée pour nourrir la population mondiale, les USA pourraient subvenir aux besoins en alimentation de 112072523.0 individus par an, soit 1/5 de la population en état de sous-nutrition.


Maintenant essayons de déterminer quelle part représente l'utilisation des céréales confondues dans l'alimentation animale aux USA

In [101]:
# recherche des informations sur les céréales tout type confondu aux USA vues dans la table_cereales

table_cereales.loc["États-Unis d'Amérique"]

Aliments pour animaux                                            140096.00
Autres Utilisations                                              137181.00
Disponibilité alimentaire (Kcal/personne/jour)                      801.00
Disponibilité alimentaire en quantité (kg/personne/an)              105.63
Disponibilité de matière grasse en quantité (g/personne/jour)         3.63
Disponibilité de protéines en quantité (g/personne/jour)             23.70
Disponibilité intérieure                                         343006.00
Exportations - Quantité                                           66302.00
Importations - Quantité                                           13477.00
Nourriture                                                        33810.00
Pertes                                                              222.00
Production                                                       433686.00
Semences                                                           3098.00
Traitement               

In [102]:
# Calcul de la part que représente l'utilisation animale des céréales par rapport à la disponibilité intérieure (aliments pour animaux/disponibilité intérieure) * 100

utilisation_animale_usa = round(( 139985 / 391316) * 100,1)
utilisation_animale_usa

35.8

In [103]:
# Conversion de l'utilisation animale (millier de tonnes) en KG pour ensuite déterminer la valeur calorifique disponible

cereales_kg = 139985 * 1000000
cereales_kg

139985000000

In [104]:
# Calcul de la moyenne de la valeur énergétique des céréales

moy_ratio_energie = (cereale['ratio_energie_poids']).mean()
moy_ratio_energie

2955.407827560563

In [105]:
# Calcul de la valeur énergétique des céréales dédiées à l'utilisation animale

cereale_animale = cereales_kg * moy_ratio_energie
cereale_animale

413712764741065.44

In [106]:
# Nombre de personnes qui pourraient être nourries pendant 1 an sur une base de 2400 kcal/j si la nourriture animale était utilisée pour nourrir la population

pers_nourrie_cereale = round((cereale_animale / 2400) / 365,1)
print("Si les USA redistribuaient la part de nourriture qu'ils allouent à l'alimentation animale aux humains, ils pourraient nourrir",pers_nourrie_cereale,"d'individus, soit 4/5 de la population mondiale en état de sous-nutrition")

Si les USA redistribuaient la part de nourriture qu'ils allouent à l'alimentation animale aux humains, ils pourraient nourrir 472274845.6 d'individus, soit 4/5 de la population mondiale en état de sous-nutrition


### 6.3 Utilisation des céréales dans l'alimentation animale au niveau mondial

Maintenant, amusons-nous à calculer le nombre de personne pouvant être nourrie si la part de céréales attribuée à l'alimentation animale mondiale était utilisée pour nourrir les Hommes

In [107]:
# Calcul de la somme des céréales utilisées pour l'alimentation animale et conversion en KG

sum_utilisation_animale = cereale['Aliments pour animaux'].sum()
sum_kilo = sum_utilisation_animale * 1000000
print(sum_kilo) 

# Calcul de la moyenne de la valeur énergétique des céréales (vu précédemment)

print(moy_ratio_energie )


823806000000.0
2955.407827560563


In [108]:
# Calcul de la disponibilité calorique de la quantité de céréales utilisée pour l'alimentation animale chaque année

apport_kcal_cereal = moy_ratio_energie * sum_kilo

# Calcul du nombre de personne qui pourrait être nourrie chaque année si la quantité de céréales dédiée à l'alimentation animale était utilisée pour nourrir la population
nbr_pers_nourrie_cereale = round((apport_kcal_cereal / 2400) / 365,0)

print("Chaque année,", sum_kilo,"KG de céréales sont utilisés pour nourrir des animaux. Si cette part était utilisée pour nourrir la population mondiale, les céréales pourraient subvenir aux besoins en alimentation de", nbr_pers_nourrie_cereale, "individus par an, soit plus d'1/3 de la population mondiale." )

Chaque année, 823806000000.0 KG de céréales sont utilisés pour nourrir des animaux. Si cette part était utilisée pour nourrir la population mondiale, les céréales pourraient subvenir aux besoins en alimentation de 2779318152.0 individus par an, soit plus d'1/3 de la population mondiale.


In [109]:
sum_sous_nutrition = round(comparaison["population_sous_nutrition"].sum() * 1000000,0)
sum_sous_nutrition

535700000.0

In [110]:
# Calcul des besoins en calorie par an pour nourrir la population en état de sous-nutrition (base de 2400k/j)

besoin_alimentaire = (sum_sous_nutrition * 2400) * 365
besoin_alimentaire

469273200000000.0

In [111]:
# Calcul du nombre de kg de ceréales que cela représente

kg_cereale_necessaire = (besoin_alimentaire / moy_ratio_energie)
kg_cereale_necessaire

158784583171.1642

In [112]:
#Calcul de la proportion des céréales utilisés pour nourrir les animaux que cela représente

proportion_cereale = round((kg_cereale_necessaire/sum_kilo) * 100,1)
print("Si au niveau mondial,", proportion_cereale,"% des céréales utilisées pour nourrir les animaux étaient redistribuées aux humains, on pourrait régler le problème de sous-nutrition")

Si au niveau mondial, 19.3 % des céréales utilisées pour nourrir les animaux étaient redistribuées aux humains, on pourrait régler le problème de sous-nutrition


### 6.4 Pertes céréalières annuelles

In [113]:
# Calcul de la somme des céréales perdues chaque année et conversion en KG

sum_perte_cereale = cereale['Pertes'].sum()
sum_perte = sum_perte_cereale * 1000000

sum_cereal = dispo_cereale * 1000000

In [114]:
# Calcul du % de céréales perdues

proportion_perdue =(sum_perte/sum_cereal) * 100
proportion_perdue

4.339553111571712

In [115]:
# Calcul de la disponibilité calorique de la quantité de céréales perdue chaque année
apport_kcal_cereal_perte = moy_ratio_energie * sum_perte

# Calcul du nombre de personne qui pourrait être nourrie chaque année si la quantité de céréales perdue était utilisée pour nourrir la population
nbr_pers_nourrie_perte = round((apport_kcal_cereal_perte / 2400) / 365,0)

print("Chaque année,", sum_perte,"KG de céréales sont perdus. Si cette part était utilisée pour nourrir la population mondiale, les céréales perdues pourraient subvenir aux besoins en alimentation de", nbr_pers_nourrie_perte, "individus par an, soit presque les 2/3 de la population en état de sous-nutrition." )

Chaque année, 104453000000.0 KG de céréales sont perdus. Si cette part était utilisée pour nourrir la population mondiale, les céréales perdues pourraient subvenir aux besoins en alimentation de 352398646.0 individus par an, soit presque les 2/3 de la population en état de sous-nutrition.
